# Exercise - Forecasting with Fundamentals

# 1. Carry Strategy for Dividend Yield

In this exercise, we consider the dividend-yield of individual stocks in order to invest in **high-carry** securities.

Note that we could consider the full analysis below with other metrics, including the PE ratio provided in the same data set.

#### Data

Use the data in `data/spx_data_weekly.xlsx`.
* `spx data` - time-series data for price, dividend-price ratio, and price-earnings ratio
* `sector data` - ETFs on sub-sectors
* `additional data` - `SPY`, `SHV`, and other benchmarks
* `rates data` - index levels for rates

Risk-free rate could be 
* `SHV`
*  `USGG3M Index` - annualized percent

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


FILE_DATA = '../data/spx_data_weekly.xlsx'
SHEET_INFO = 'spx names'

info = pd.read_excel(FILE_DATA, sheet_name=SHEET_INFO)
info.set_index('ticker',inplace=True)
info.rename(columns={'security_name':'security name'},inplace=True)

display(info)

,name,gics_sector_name,mkt cap
ticker,,,
A,Agilent Technologies Inc,Health Care,4.162353e+10
AAPL,Apple Inc,Information Technology,4.025226e+12
ABBV,AbbVie Inc,Health Care,4.106695e+11
ABNB,Airbnb Inc,Consumer Discretionary,7.510938e+10
ABT,Abbott Laboratories,Health Care,2.270793e+11
...,...,...,...
XYZ,Block Inc,Financials,3.675158e+10
YUM,Yum! Brands Inc,Consumer Discretionary,4.137582e+10
ZBH,Zimmer Biomet Holdings Inc,Health Care,1.781850e+10


In [2]:
def load_ts(file_data,sheet_name):
    ts = pd.read_excel(file_data,sheet_name=sheet_name,
                    header=[0,1],                  
                    index_col=0)
                    
    ts.columns.names = ["ticker","field"]

    ts.rename(columns={
        'PX_LAST':'price',
        'EQY_DVD_YLD_IND':'dvd yld',
        'PE_RATIO':'pe ratio'
    }, level=1, inplace=True)

    return ts

In [3]:
SHEET_TS = 'spx data'
spx = load_ts(FILE_DATA,SHEET_TS)
display(spx.tail().style.format('{:.2f}',na_rep='').format_index(lambda x: x.strftime('%Y-%m-%d')))

### 1.0. Data Processing

Filter the data to drop any ticker which does not have at least `5` years of continuous price data.

In [4]:
MIN_OBS = 5 * 52 # data is weekly (5 years of continuous data)

prices = spx.loc[:,(slice(None),'price')]

def has_continuous_data(series, min_weeks):
    """
    Check if a series has at least min_weeks of continuous (non-null) data.
    Returns True if there exists a continuous period of at least min_weeks.
    """
    # Drop null values and sort by date
    non_null = series.dropna().sort_index()
    
    if len(non_null) < min_weeks:
        return False
    
    # Calculate time differences between consecutive observations
    # For weekly data, gaps > 14 days indicate a break in continuity
    date_diffs = non_null.index.to_series().diff()
    
    # Identify gaps (more than 2 weeks between observations)
    gaps = date_diffs > pd.Timedelta(days=14)
    
    # If no gaps, the entire series is continuous
    if not gaps.any():
        return len(non_null) >= min_weeks
    
    # Find all continuous segments
    gap_indices = non_null.index[gaps]
    
    # Check each continuous segment
    start = 0
    for gap_idx in gap_indices:
        segment_end = non_null.index.get_loc(gap_idx)
        segment_length = segment_end - start
        if segment_length >= min_weeks:
            return True
        start = segment_end + 1
    
    # Check the final segment
    final_segment_length = len(non_null) - start
    if final_segment_length >= min_weeks:
        return True
    
    return False

# Check each ticker for continuous data
valid_tickers = []
for ticker in prices.columns.get_level_values(0).unique():
    ticker_prices = prices[ticker]
    if has_continuous_data(ticker_prices, MIN_OBS):
        valid_tickers.append(ticker)

historic_tickers = pd.Index(valid_tickers)

spx = spx[historic_tickers]
info = info.loc[historic_tickers]

spx.head()

ticker            A                      AAPL                      ABBV  \
field         price dvd yld pe ratio    price dvd yld pe ratio    price   
date                                                                      
2015-07-03  36.4044  1.0988  27.4720  28.2589  1.8401  13.0592  44.2648   
2015-07-10  36.2388  1.1038  27.3471  27.5527  1.8873  12.7328  44.9267   
2015-07-17  36.7447  1.0886  27.7288  28.9696  1.7950  13.3876  45.7570   
2015-07-24  36.1560  1.1063  27.2846  27.8253  1.8688  12.8588  44.5083   
2015-07-31  37.6645  1.0620  26.6192  27.1101  1.9181  12.5283  45.7701   

ticker                           ABT  ...      YUM      ZBH                   \
field      dvd yld pe ratio    price  ... pe ratio    price dvd yld pe ratio   
date                                  ...                                      
2015-07-03  4.6086  11.9239  40.4951  ...  18.6689  96.5853  0.9111  18.2975   
2015-07-10  4.5407  12.1022  40.9213  ...  18.5282  95.3533  0.9229  18.0642   
2015-07-17  4.4583  12.3258  41.0942  ...  18.0008  95.9693  0.9170  18.1809   
2015-07-24  4.5834  11.9895  42.0413  ...  17.7611  95.2194  0.9242  18.0388   
2015-07-31  4.4571  12.3294  41.7448  ...  17.9762  92.9072  0.9472  17.6008   

ticker        ZBRA                        ZTS                   
field        price dvd yld  pe ratio    price dvd yld pe ratio  
date                                                            
2015-07-03  112.43     NaN  151.5166  44.5029  0.7460  26.8168  
2015-07-10  109.75     NaN  147.9049  43.2593  0.7675  26.0674  
2015-07-17  114.18     NaN  153.8750  44.0331  0.7540  26.5337  
2015-07-24  110.89     NaN  149.4412  45.5900  0.7282  27.4718  
2015-07-31  107.63     NaN  145.0478  45.1201  0.7358  27.1887  

[5 rows x 1455 columns]

### 1.1. Data Processing

Report the highest and lowest dividend-yielding stocks...

* for any given date across the entire panel
* taking an average of the past year

Which stocks were they? Was it driven more by changes in D or P?

In [5]:
# Extract dividend yield and price data
dvd_yld = spx.loc[:,(slice(None),'dvd yld')].droplevel(1, axis=1)
prices = spx.loc[:,(slice(None),'price')].droplevel(1, axis=1)

# 1. For any given date across the entire panel
# Find highest and lowest dividend-yielding stocks at each date
dividend_stats_per_date = []

for date in dvd_yld.index:
    date_yields = dvd_yld.loc[date].dropna()
    if len(date_yields) > 0:
        max_yield = date_yields.max()
        min_yield = date_yields.min()
        max_stock = date_yields.idxmax()
        min_stock = date_yields.idxmin()
        
        dividend_stats_per_date.append({
            'Date': date,
            'Max Yield': max_yield,
            'Min Yield': min_yield,
            'Max Stock': max_stock,
            'Min Stock': min_stock
        })

dividend_stats_df = pd.DataFrame(dividend_stats_per_date).set_index('Date')

print("=== Highest and Lowest Dividend-Yielding Stocks by Date ===")
print(f"\nExample: First few dates")
display(dividend_stats_df.head(10))

print(f"\nMost frequent highest yield stock: {dividend_stats_df['Max Stock'].mode()[0]}")
print(f"Most frequent lowest yield stock: {dividend_stats_df['Min Stock'].mode()[0]}")

# 2. Taking an average of the past year (52 weeks)
# Calculate rolling 1-year average of dividend yield
rolling_avg_yield = dvd_yld.rolling(window=52, min_periods=1).mean()

# Find highest and lowest based on rolling average
rolling_stats = []

for date in rolling_avg_yield.index:
    date_avg_yields = rolling_avg_yield.loc[date].dropna()
    if len(date_avg_yields) > 0:
        max_avg_yield = date_avg_yields.max()
        min_avg_yield = date_avg_yields.min()
        max_stock_avg = date_avg_yields.idxmax()
        min_stock_avg = date_avg_yields.idxmin()
        
        rolling_stats.append({
            'Date': date,
            'Max Avg Yield': max_avg_yield,
            'Min Avg Yield': min_avg_yield,
            'Max Stock (Avg)': max_stock_avg,
            'Min Stock (Avg)': min_stock_avg
        })

rolling_stats_df = pd.DataFrame(rolling_stats).set_index('Date')

print("\n=== Highest and Lowest Dividend-Yielding Stocks (1-Year Rolling Average) ===")
print(f"\nExample: First few dates with sufficient data")
# Show dates after we have at least 52 weeks of data
display(rolling_stats_df.iloc[52:62])

print(f"\nMost frequent highest yield stock (rolling avg): {rolling_stats_df['Max Stock (Avg)'].mode()[0]}")
print(f"Most frequent lowest yield stock (rolling avg): {rolling_stats_df['Min Stock (Avg)'].mode()[0]}")

# 3. Analyze whether driven by D (dividend) or P (price)
# Dividend yield = D/P, so D = P * (dvd yld / 100)
# Calculate implied dividends (dividend yield is in percentage)
implied_dividends = prices * (dvd_yld / 100)  # Convert yield % to decimal

# Calculate year-over-year changes
# For each stock, calculate changes in D, P, and yield
def analyze_drivers(dvd_yld_series, price_series, dividend_series, window=52):
    """
    Analyze whether yield changes are driven by D or P changes.
    Returns correlation between yield changes and D changes vs P changes.
    """
    # Calculate rolling averages
    avg_yield = dvd_yld_series.rolling(window=window).mean()
    avg_price = price_series.rolling(window=window).mean()
    avg_dividend = dividend_series.rolling(window=window).mean()
    
    # Calculate changes (year-over-year)
    yield_change = avg_yield.pct_change(52)
    price_change = avg_price.pct_change(52)
    dividend_change = avg_dividend.pct_change(52)
    
    # Combine into dataframe
    changes_df = pd.DataFrame({
        'yield_change': yield_change,
        'price_change': price_change,
        'dividend_change': dividend_change
    }).dropna()
    
    if len(changes_df) < 10:
        return None, None, None
    
    # Calculate correlations
    corr_yield_price = changes_df['yield_change'].corr(changes_df['price_change'])
    corr_yield_dividend = changes_df['yield_change'].corr(changes_df['dividend_change'])
    
    return corr_yield_price, corr_yield_dividend, changes_df

# Analyze for stocks that appear frequently as highest/lowest yielders
top_stocks = rolling_stats_df['Max Stock (Avg)'].value_counts().head(5).index
bottom_stocks = rolling_stats_df['Min Stock (Avg)'].value_counts().head(5).index

print("\n=== Analysis: Driven by D or P? ===")
print("\nFor highest dividend-yielding stocks (top 5 most frequent):")
for stock in top_stocks:
    if stock in dvd_yld.columns and stock in prices.columns:
        corr_p, corr_d, changes = analyze_drivers(
            dvd_yld[stock], prices[stock], implied_dividends[stock]
        )
        if corr_p is not None:
            print(f"\n{stock}:")
            print(f"  Correlation (yield change vs price change): {corr_p:.3f}")
            print(f"  Correlation (yield change vs dividend change): {corr_d:.3f}")
            if abs(corr_p) > abs(corr_d):
                print(f"  → More driven by PRICE changes")
            else:
                print(f"  → More driven by DIVIDEND changes")

print("\nFor lowest dividend-yielding stocks (top 5 most frequent):")
for stock in bottom_stocks:
    if stock in dvd_yld.columns and stock in prices.columns:
        corr_p, corr_d, changes = analyze_drivers(
            dvd_yld[stock], prices[stock], implied_dividends[stock]
        )
        if corr_p is not None:
            print(f"\n{stock}:")
            print(f"  Correlation (yield change vs price change): {corr_p:.3f}")
            print(f"  Correlation (yield change vs dividend change): {corr_d:.3f}")
            if abs(corr_p) > abs(corr_d):
                print(f"  → More driven by PRICE changes")
            else:
                print(f"  → More driven by DIVIDEND changes")

# Overall analysis across all stocks
print("\n=== Overall analysis across all stocks ===")
all_corrs_p = []
all_corrs_d = []

for stock in dvd_yld.columns:
    if stock in prices.columns:
        corr_p, corr_d, _ = analyze_drivers(
            dvd_yld[stock], prices[stock], implied_dividends[stock]
        )
        if corr_p is not None and not pd.isna(corr_p) and not pd.isna(corr_d):
            all_corrs_p.append(corr_p)
            all_corrs_d.append(corr_d)

if len(all_corrs_p) > 0:
    avg_corr_p = np.mean(all_corrs_p)
    avg_corr_d = np.mean(all_corrs_d)
    print(f"\nAverage correlation (yield change vs price change): {avg_corr_p:.3f}")
    print(f"Average correlation (yield change vs dividend change): {avg_corr_d:.3f}")
    
    if abs(avg_corr_p) > abs(avg_corr_d):
        print(f"\n→ Overall, dividend yield changes are MORE driven by PRICE changes")
    else:
        print(f"\n→ Overall, dividend yield changes are MORE driven by DIVIDEND changes")


=== Highest and Lowest Dividend-Yielding Stocks by Date ===

Example: First few dates


,Max Yield,Min Yield,Max Stock,Min Stock
Date,,,,
2015-07-03,20.8790,0.0270,DD,CI
2015-07-10,20.8668,0.0277,DD,CI
2015-07-17,20.9647,0.0284,DD,CI
2015-07-24,23.3362,0.0299,DD,CI
2015-07-31,22.7957,0.0302,DD,CI
2015-08-07,23.5772,0.0298,DD,CI
2015-08-14,23.8923,0.0302,DD,CI
2015-08-21,25.1704,0.0305,DD,CI
2015-08-28,24.3810,0.0316,DD,CI



Most frequent highest yield stock: MO
Most frequent lowest yield stock: COO

=== Highest and Lowest Dividend-Yielding Stocks (1-Year Rolling Average) ===

Example: First few dates with sufficient data


,Max Avg Yield,Min Avg Yield,Max Stock (Avg),Min Stock (Avg)
Date,,,,
2016-07-01,23.145127,0.031708,DD,CI
2016-07-08,23.184600,0.031821,DD,CI
2016-07-15,23.199671,0.031912,DD,CI
2016-07-22,23.160790,0.031910,DD,CI
2016-07-29,23.128104,0.031977,DD,CI
2016-08-05,23.080615,0.032056,DD,CI
2016-08-12,23.037229,0.032102,DD,CI
2016-08-19,22.960773,0.032144,DD,CI
2016-08-26,22.897902,0.032192,DD,CI



Most frequent highest yield stock (rolling avg): DD
Most frequent lowest yield stock (rolling avg): COO

=== Analysis: Driven by D or P? ===

For highest dividend-yielding stocks (top 5 most frequent):

DD:
  Correlation (yield change vs price change): -0.033
  Correlation (yield change vs dividend change): 0.742
  → More driven by DIVIDEND changes

MO:
  Correlation (yield change vs price change): -0.950
  Correlation (yield change vs dividend change): 0.776
  → More driven by PRICE changes

T:
  Correlation (yield change vs price change): -0.502
  Correlation (yield change vs dividend change): 0.748
  → More driven by DIVIDEND changes

KDP:
  Correlation (yield change vs price change): -0.838
  Correlation (yield change vs dividend change): 0.947
  → More driven by DIVIDEND changes

OKE:
  Correlation (yield change vs price change): -0.906
  Correlation (yield change vs dividend change): 0.235
  → More driven by PRICE changes

For lowest dividend-yielding stocks (top 5 most frequent

### 1.2. A Carry Strategy

For this strategy, use the `dvd yld` data.

Build a portfolio where at every time $t$, you set weights, $w_t$ that invest...
* long the highest ranking `20%` stocks.

For now, go equal weights of `0.01` in any stock for which you're long or short. 

#### Realized Returns
Start at the beginning of the sample.
* At time $t$, rank the stocks. 
* The selected stocks will earn $r_{t+1}$.

In [6]:
# Extract dividend yield and price data
dvd_yld = spx.loc[:,(slice(None),'dvd yld')].droplevel(1, axis=1)
prices = spx.loc[:,(slice(None),'price')].droplevel(1, axis=1)

# Calculate returns: r_t = P_t / P_{t-1} - 1
returns = prices.pct_change()

# Portfolio construction
# At each time t, rank stocks by dividend yield and go long top 20%
# The portfolio earns r_t (based on weights set at t-1 using yields at t-1)
WEIGHT_PER_STOCK = 0.01
TOP_PCT = 0.20

ls_pf_rets = []
lspf_weights_history = []

for t in range(1, len(dvd_yld.index)):  # Start at 1, not 0
    date_t = dvd_yld.index[t]
    date_t_prev = dvd_yld.index[t - 1]
    
    # Get dividend yields at time t-1 to set weights for period t
    yields_t_prev = dvd_yld.loc[date_t_prev].dropna()
    
    if len(yields_t_prev) == 0:
        ls_pf_rets.append(np.nan)
        lspf_weights_history.append({})
        continue
    
    # Rank stocks by dividend yield (highest first)
    # Calculate how many stocks to select (top 20%)
    n_stocks = len(yields_t_prev)
    n_long = max(1, int(np.ceil(n_stocks * TOP_PCT)))
    
    # Select top 20% by dividend yield
    top_stocks = yields_t_prev.nlargest(n_long).index
    
    # Create weight vector: 0.01 for selected stocks, 0 otherwise
    weights = pd.Series(0.0, index=yields_t_prev.index)
    weights[top_stocks] = WEIGHT_PER_STOCK
    
    # Get returns for current period (r_t)
    returns_t = returns.loc[date_t]
    
    # Calculate portfolio return: sum of weights * returns
    # Only use stocks that have both weights and returns
    valid_stocks = weights.index.intersection(returns_t.dropna().index)
    if len(valid_stocks) > 0:
        portfolio_return = (weights[valid_stocks] * returns_t[valid_stocks]).sum()
    else:
        portfolio_return = np.nan
    
    ls_pf_rets.append(portfolio_return)
    lspf_weights_history.append(weights.to_dict())

# Create portfolio returns series
portfolio_returns_series = pd.Series(
    ls_pf_rets, 
    index=dvd_yld.index[1:]  # Start from index 1, no return for first period
)

print("=== Carry strategy portfolio (long top 20% by dividend yield) ===")
print(f"\nPortfolio Statistics:")
print(f"  Total periods: {len(portfolio_returns_series.dropna())}")
print(f"  Mean return: {portfolio_returns_series.mean():.4f} per period")
print(f"  Annualized mean: {portfolio_returns_series.mean() * 52:.4f}")
print(f"  Volatility: {portfolio_returns_series.std():.4f} per period")
print(f"  Annualized volatility: {portfolio_returns_series.std() * np.sqrt(52):.4f}")

print(f"\nFirst 10 portfolio returns:")
display(portfolio_returns_series.head(10))

print(f"\nLast 10 portfolio returns:")
display(portfolio_returns_series.tail(10))

# Store for later use
long_only_returns = portfolio_returns_series

=== Carry strategy portfolio (long top 20% by dividend yield) ===

Portfolio Statistics:
  Total periods: 521
  Mean return: 0.0022 per period
  Annualized mean: 0.1125
  Volatility: 0.0227 per period
  Annualized volatility: 0.1640

First 10 portfolio returns:


date
2015-07-10    0.004991
2015-07-17    0.004529
2015-07-24   -0.017008
2015-07-31    0.016891
2015-08-07   -0.005494
2015-08-14    0.011324
2015-08-21   -0.029137
2015-08-28   -0.007640
2015-09-04   -0.030645
2015-09-11    0.009234
dtype: float64


Last 10 portfolio returns:


date
2025-04-25    0.017512
2025-05-02    0.017658
2025-05-09   -0.000523
2025-05-16    0.028888
2025-05-23   -0.030130
2025-05-30    0.011345
2025-06-06    0.005587
2025-06-13    0.000594
2025-06-20   -0.002336
2025-06-27    0.008726
dtype: float64

### 1.3. Long-Short

Re-do the strategy of the previous section, but this time go long and short...

At any given time $t$, set the vector $w_t$ as follows...
* long the highest-ranking `20%` of stocks.
* short the lowest-ranking `20%` of stocks.
* take no position in all other stocks.

In [7]:
WEIGHT_PER_STOCK = 0.01
TOP_PCT = 0.20
BOTTOM_PCT = 0.20

ls_pf_rets = []
lspf_weights_history = []

for t in range(1, len(dvd_yld.index)):  # Start at 1, not 0
    date_t = dvd_yld.index[t]
    date_t_prev = dvd_yld.index[t - 1]
    
    # Get dividend yields at time t-1 to set weights for period t
    yields_t_prev = dvd_yld.loc[date_t_prev].dropna()
    
    if len(yields_t_prev) == 0:
        ls_pf_rets.append(np.nan)
        lspf_weights_history.append({})
        continue
    
    # Rank stocks by dividend yield (highest first)
    # Calculate how many stocks to select (top 20%)
    n_stocks = len(yields_t_prev)
    n_long = max(1, int(np.ceil(n_stocks * TOP_PCT)))
    n_short = max(1, int(np.ceil(n_stocks * BOTTOM_PCT)))
    
    # Select top 20% by dividend yield
    top_stocks = yields_t_prev.nlargest(n_long).index
    bottom_stocks = yields_t_prev.nsmallest(n_short).index
    
    # Create weight vector: 0.01 for selected stocks, 0 otherwise
    weights = pd.Series(0.0, index=yields_t_prev.index)
    weights[top_stocks] = WEIGHT_PER_STOCK
    weights[bottom_stocks] = -WEIGHT_PER_STOCK
    # Get returns for current period (r_t)
    returns_t = returns.loc[date_t]
    
    # Calculate portfolio return: sum of weights * returns
    # Only use stocks that have both weights and returns
    valid_stocks = weights.index.intersection(returns_t.dropna().index)
    if len(valid_stocks) > 0:
        portfolio_return = (weights[valid_stocks] * returns_t[valid_stocks]).sum()
    else:
        portfolio_return = np.nan
    
    ls_pf_rets.append(portfolio_return)
    lspf_weights_history.append(weights.to_dict())

# Create portfolio returns series
ls_portfolio_returns = pd.Series(
    ls_pf_rets, 
    index=dvd_yld.index[1:]  # Start from index 1, no return for first period
)

print("=== Carry strategy portfolio (long-short top-bottom 20% by dividend yield) ===")
print(f"\nPortfolio Statistics:")
print(f"  Total periods: {len(ls_portfolio_returns.dropna())}")
print(f"  Mean return: {ls_portfolio_returns.mean():.4f} per period")
print(f"  Annualized mean: {ls_portfolio_returns.mean() * 52:.4f}")
print(f"  Volatility: {ls_portfolio_returns.std():.4f} per period")
print(f"  Annualized volatility: {ls_portfolio_returns.std() * np.sqrt(52):.4f}")

print(f"\nFirst 10 portfolio returns:")
display(ls_portfolio_returns.head(10))

print(f"\nLast 10 portfolio returns:")
display(ls_portfolio_returns.tail(10))

# Store for later use
long_short_returns = ls_portfolio_returns

=== Carry strategy portfolio (long-short top-bottom 20% by dividend yield) ===

Portfolio Statistics:
  Total periods: 521
  Mean return: -0.0001 per period
  Annualized mean: -0.0056
  Volatility: 0.0133 per period
  Annualized volatility: 0.0957

First 10 portfolio returns:


date
2015-07-10    0.004285
2015-07-17   -0.001843
2015-07-24   -0.005811
2015-07-31    0.005861
2015-08-07   -0.000700
2015-08-14    0.002477
2015-08-21    0.010438
2015-08-28   -0.008036
2015-09-04   -0.011229
2015-09-11   -0.010142
dtype: float64


Last 10 portfolio returns:


date
2025-04-25   -0.014733
2025-05-02   -0.007050
2025-05-09   -0.000867
2025-05-16   -0.010925
2025-05-23   -0.008069
2025-05-30    0.001414
2025-06-06   -0.007252
2025-06-13    0.012516
2025-06-20   -0.003675
2025-06-27   -0.016824
dtype: float64

### 1.4. Performance

Calculate the return of your portfolio over time. 

Report the following annualized stats
* mean
* volatility
* Sharpe 

Also calculate the tail-risk stats
* skewness
* VaR (5th quantile)
* CVaR (5th quantile)
* max drawdown

For both tables, compare to `SPY`, found in the `additional data` tab.

In [8]:
SHEET_BENCH = 'additional data'
bench = load_ts(FILE_DATA,SHEET_BENCH)
display(bench[['SPY']].tail().style.format('{:.2f}',na_rep='').format_index('{:%Y-%m-%d}'))

ticker,SPY
field,price
date,
2025-05-30,586.03
2025-06-06,595.72
2025-06-13,593.59
2025-06-20,592.64
2025-06-27,613.21


In [9]:
def performance_metrics(returns,annualization=1, quantile=.05):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()
    return metrics

def maximum_drawdown(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max

    max_drawdown = drawdown.min()
    end_date = drawdown.idxmin()
    summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})

    for col in drawdown:
        summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
        recovery = (drawdown.loc[end_date[col]:,col])
        try:
            summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
        except:
            summary.loc[col,'Recover'] = pd.to_datetime(None)

        summary['Peak'] = pd.to_datetime(summary['Peak'])
        try:
            summary['Duration (to Recover)'] = (summary['Recover'] - summary['Peak'])
        except:
            summary['Duration (to Recover)'] = None
            
        summary = summary[['Max Drawdown','Peak','Bottom','Recover','Duration (to Recover)']]

    return summary    


def tail_metrics(returns, quantile=.05, relative=False, mdd=True):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Skewness'] = returns.skew()
    metrics['Kurtosis'] = returns.kurtosis()

    VaR = returns.quantile(quantile)
    CVaR = (returns[returns < returns.quantile(quantile)]).mean()

    if relative:
        VaR = (VaR - returns.mean())/returns.std()
        CVaR = (CVaR - returns.mean())/returns.std()

    metrics[f'VaR ({quantile})'] = VaR
    metrics[f'CVaR ({quantile})'] = CVaR

    if mdd:
        mdd_stats = maximum_drawdown(returns)
        metrics = metrics.join(mdd_stats)

        if relative:
            metrics['Max Drawdown'] = (metrics['Max Drawdown'] - returns.mean())/returns.std()

    return metrics

In [10]:
# Extract SPY returns
spy_prices = bench.loc[:, ('SPY', 'price')]
spy_returns = spy_prices.pct_change()

# Align dates between portfolios and SPY
# Use the intersection of dates
common_dates = long_short_returns.index.intersection(spy_returns.index)
ls_returns_aligned = long_short_returns.loc[common_dates]
lo_returns_aligned = long_only_returns.loc[common_dates]
spy_returns_aligned = spy_returns.loc[common_dates]

# Combine into a DataFrame for analysis
all_returns = pd.DataFrame({
    'Long-Short': ls_returns_aligned,
    'Long-Only': lo_returns_aligned,
    'SPY': spy_returns_aligned
})

# Calculate annualized performance metrics
ANNUALIZATION = 52  # Weekly data

print("=" * 80)
print("ANNUALIZED PERFORMANCE METRICS")
print("=" * 80)

perf_stats = performance_metrics(all_returns, annualization=ANNUALIZATION, quantile=0.05)
display(perf_stats.style.format('{:.4f}'))

# Calculate tail-risk statistics
print("\n" + "=" * 80)
print("TAIL-RISK STATISTICS")
print("=" * 80)

tail_stats = tail_metrics(all_returns, quantile=0.05, relative=False, mdd=True)
display(tail_stats.style.format({
    'Skewness': '{:.4f}',
    'Kurtosis': '{:.4f}',
    'VaR (0.05)': '{:.4f}',
    'CVaR (0.05)': '{:.4f}',
    'Max Drawdown': '{:.4f}'
}))

print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)
print("\nAnnualized Performance:")
print(f"  - Long-Short: Mean={perf_stats.loc['Long-Short', 'Mean']:.2%}, Vol={perf_stats.loc['Long-Short', 'Vol']:.2%}, Sharpe={perf_stats.loc['Long-Short', 'Sharpe']:.2f}")
print(f"  - Long-Only:  Mean={perf_stats.loc['Long-Only', 'Mean']:.2%}, Vol={perf_stats.loc['Long-Only', 'Vol']:.2%}, Sharpe={perf_stats.loc['Long-Only', 'Sharpe']:.2f}")
print(f"  - SPY:        Mean={perf_stats.loc['SPY', 'Mean']:.2%}, Vol={perf_stats.loc['SPY', 'Vol']:.2%}, Sharpe={perf_stats.loc['SPY', 'Sharpe']:.2f}")

print("\nTail Risk Comparison:")
print(f"  - Long-Short: Skew={tail_stats.loc['Long-Short', 'Skewness']:.2f}, MaxDD={tail_stats.loc['Long-Short', 'Max Drawdown']:.2%}, VaR={tail_stats.loc['Long-Short', 'VaR (0.05)']:.2%}")
print(f"  - Long-Only:  Skew={tail_stats.loc['Long-Only', 'Skewness']:.2f}, MaxDD={tail_stats.loc['Long-Only', 'Max Drawdown']:.2%}, VaR={tail_stats.loc['Long-Only', 'VaR (0.05)']:.2%}")
print(f"  - SPY:        Skew={tail_stats.loc['SPY', 'Skewness']:.2f}, MaxDD={tail_stats.loc['SPY', 'Max Drawdown']:.2%}, VaR={tail_stats.loc['SPY', 'VaR (0.05)']:.2%}")

ANNUALIZED PERFORMANCE METRICS


,Mean,Vol,Sharpe,Min,Max
Long-Short,-0.0056,0.0957,-0.0587,-0.0525,0.0978
Long-Only,0.1125,0.1640,0.6860,-0.1508,0.1894
SPY,0.1408,0.1732,0.8131,-0.1455,0.1209



TAIL-RISK STATISTICS


,Skewness,Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown,Peak,Bottom,Recover,Duration (to Recover)
Long-Short,1.0590,8.4382,-0.0180,-0.0279,-0.2442,2016-12-30 00:00:00,2020-11-06 00:00:00,2022-05-20 00:00:00,1967 days 00:00:00
Long-Only,0.2683,18.4711,-0.0291,-0.0508,-0.3610,2020-01-17 00:00:00,2020-03-20 00:00:00,2020-11-27 00:00:00,315 days 00:00:00
SPY,-0.5950,5.8815,-0.0336,-0.0577,-0.3183,2020-02-14 00:00:00,2020-03-20 00:00:00,2020-08-07 00:00:00,175 days 00:00:00



INTERPRETATION

Annualized Performance:
  - Long-Short: Mean=-0.56%, Vol=9.57%, Sharpe=-0.06
  - Long-Only:  Mean=11.25%, Vol=16.40%, Sharpe=0.69
  - SPY:        Mean=14.08%, Vol=17.32%, Sharpe=0.81

Tail Risk Comparison:
  - Long-Short: Skew=1.06, MaxDD=-24.42%, VaR=-1.80%
  - Long-Only:  Skew=0.27, MaxDD=-36.10%, VaR=-2.91%
  - SPY:        Skew=-0.60, MaxDD=-31.83%, VaR=-3.36%


***

# 2. Attribution

### 2.1. Market Exposure

For both the long-only (LO) and long-short (LS) strategies, estimate a linear factor decomposition (LFD) against `SPY`

Report the...
* alpha (annualized)
* beta
* r-squared

Also show the correlation matrix of the strategies.

How do the LO and LS strategies compare to `SPY`?

In [11]:
from sklearn.linear_model import LinearRegression
from scipy import stats as scipy_stats

# Use the aligned returns from previous section
# Regression: portfolio_return = alpha + beta * SPY_return + epsilon

print("=" * 80)
print("LINEAR FACTOR DECOMPOSITION (LFD) - MARKET EXPOSURE")
print("=" * 80)

# Prepare data
X = all_returns[['SPY']].values  # SPY returns as independent variable
strategies = ['Long-Short', 'Long-Only']

# Store results
lfd_results = pd.DataFrame(
    index=strategies,
    columns=['Alpha (annualized)', 'Beta', 'R-squared', 't-stat (alpha)', 'p-value (alpha)']
)

for strategy in strategies:
    # Get strategy returns
    y = all_returns[strategy].values.reshape(-1, 1)
    
    # Remove NaN values
    mask = ~(np.isnan(X).any(axis=1) | np.isnan(y).ravel())
    X_clean = X[mask]
    y_clean = y[mask]
    
    # Run regression
    model = LinearRegression()
    model.fit(X_clean, y_clean)
    
    # Get coefficients
    alpha = model.intercept_[0]
    beta = model.coef_[0][0]
    
    # Calculate R-squared
    y_pred = model.predict(X_clean)
    ss_res = np.sum((y_clean - y_pred) ** 2)
    ss_tot = np.sum((y_clean - np.mean(y_clean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    # Calculate t-statistic and p-value for alpha
    n = len(y_clean)
    residuals = y_clean - y_pred
    mse = np.sum(residuals ** 2) / (n - 2)
    se_alpha = np.sqrt(mse * (1/n + np.mean(X_clean)**2 / np.sum((X_clean - np.mean(X_clean))**2)))
    t_stat = alpha / se_alpha
    p_value = 2 * (1 - scipy_stats.t.cdf(abs(t_stat), n - 2))
    
    # Store results
    lfd_results.loc[strategy, 'Alpha (annualized)'] = alpha * ANNUALIZATION
    lfd_results.loc[strategy, 'Beta'] = beta
    lfd_results.loc[strategy, 'R-squared'] = r_squared
    lfd_results.loc[strategy, 't-stat (alpha)'] = t_stat
    lfd_results.loc[strategy, 'p-value (alpha)'] = p_value

print("\nRegression Results: Strategy = alpha + beta * SPY + epsilon")
print()
display(lfd_results.astype(float).style.format({
    'Alpha (annualized)': '{:.4f}',
    'Beta': '{:.4f}',
    'R-squared': '{:.4f}',
    't-stat (alpha)': '{:.2f}',
    'p-value (alpha)': '{:.4f}'
}))

# Correlation matrix
print("\n" + "=" * 80)
print("CORRELATION MATRIX")
print("=" * 80)

corr_matrix = all_returns[['Long-Short', 'Long-Only', 'SPY']].corr()
display(corr_matrix.style.format('{:.4f}').background_gradient(cmap='RdYlGn', vmin=-1, vmax=1))

# Interpretation
print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

for strategy in strategies:
    alpha_ann = lfd_results.loc[strategy, 'Alpha (annualized)']
    beta = lfd_results.loc[strategy, 'Beta']
    r2 = lfd_results.loc[strategy, 'R-squared']
    p_val = lfd_results.loc[strategy, 'p-value (alpha)']
    
    print(f"\n{strategy}:")
    print(f"  - Alpha: {alpha_ann:.2%} annualized {'(significant)' if p_val < 0.05 else '(not significant at 5%)'}")
    print(f"  - Beta: {beta:.2f} (market exposure)")
    print(f"  - R²: {r2:.2%} (proportion of variance explained by SPY)")
    
    if abs(beta) < 0.3:
        print(f"  → Low market exposure - relatively market-neutral")
    elif abs(beta) > 0.8:
        print(f"  → High market exposure - moves closely with SPY")
    else:
        print(f"  → Moderate market exposure")

print("\nComparison to SPY:")
print(f"  - Long-Short correlation with SPY: {corr_matrix.loc['Long-Short', 'SPY']:.2f}")
print(f"  - Long-Only correlation with SPY: {corr_matrix.loc['Long-Only', 'SPY']:.2f}")
print(f"  - Long-Short vs Long-Only correlation: {corr_matrix.loc['Long-Short', 'Long-Only']:.2f}")

LINEAR FACTOR DECOMPOSITION (LFD) - MARKET EXPOSURE

Regression Results: Strategy = alpha + beta * SPY + epsilon



,Alpha (annualized),Beta,R-squared,t-stat (alpha),p-value (alpha)
Long-Short,-0.0011,-0.0319,0.0033,-0.04,0.9702
Long-Only,0.0047,0.7653,0.6536,0.15,0.8787



CORRELATION MATRIX


,Long-Short,Long-Only,SPY
Long-Short,1.0000,0.4975,-0.0576
Long-Only,0.4975,1.0000,0.8085
SPY,-0.0576,0.8085,1.0000



INTERPRETATION

Long-Short:
  - Alpha: -0.11% annualized (not significant at 5%)
  - Beta: -0.03 (market exposure)
  - R²: 0.33% (proportion of variance explained by SPY)
  → Low market exposure - relatively market-neutral

Long-Only:
  - Alpha: 0.47% annualized (not significant at 5%)
  - Beta: 0.77 (market exposure)
  - R²: 65.36% (proportion of variance explained by SPY)
  → Moderate market exposure

Comparison to SPY:
  - Long-Short correlation with SPY: -0.06
  - Long-Only correlation with SPY: 0.81
  - Long-Short vs Long-Only correlation: 0.50


### 2.2. Sector Regression

Estimate a multivariate LFD for both LO and LS against all the sector ETFs.
* Note that `SHV` is not a sector ETF but rather a money-market fund. Exclude it.
* Exclude `SPY`.

Report the same stats as in `2.1.`

In [12]:
SHEET_SECTOR_INFO = 'sector names'
info_sectors = pd.read_excel(FILE_DATA,sheet_name=SHEET_SECTOR_INFO).rename(columns={'Unnamed: 0':'ticker'}).set_index('ticker').drop(columns=['gics_sector_name'])
info_sectors['cur_mkt_cap'] = info_sectors['cur_mkt_cap'].astype(float) / 1e9
info_sectors.rename(columns={'cur_mkt_cap':'fund size ($ billions)'},inplace=True)
display(info_sectors.style.format({'fund size ($ billions)':'${:,.2f}'}))

,security_name,fund size ($ billions)
ticker,,
XLB US Equity,Materials Select Sector SPDR F,$5.13
XLC US Equity,Communication Services Select,$25.31
XLE US Equity,Energy Select Sector SPDR Fund,$28.20
XLF US Equity,Financial Select Sector SPDR F,$52.73
XLI US Equity,Industrial Select Sector SPDR,$23.76
XLK US Equity,Technology Select Sector SPDR,$93.26
XLP US Equity,Consumer Staples Select Sector,$15.05
XLRE US Equity,Real Estate Select Sector SPDR,$7.54
XLU US Equity,Utilities Select Sector SPDR F,$22.19


In [13]:
SHEET_SECTORS = 'sector data'
sectors = load_ts(FILE_DATA,SHEET_SECTORS)
display(sectors.tail().style.format('{:.1f}',na_rep='').format_index(lambda x: x.strftime('%Y-%m-%d')))

In [14]:
# Extract sector returns
sector_prices = sectors.loc[:, (slice(None), 'price')].droplevel(1, axis=1)
sector_returns = sector_prices.pct_change()

# Exclude SHV (money market) and SPY
sector_tickers = [col for col in sector_returns.columns if col not in ['SHV', 'SPY']]
sector_returns_clean = sector_returns[sector_tickers]

print("=" * 80)
print("MULTIVARIATE LFD - SECTOR EXPOSURE")
print("=" * 80)
print(f"\nUsing {len(sector_tickers)} sector ETFs: {', '.join(sector_tickers)}")

# Align dates between portfolios and sectors
common_dates_sectors = long_short_returns.index.intersection(sector_returns_clean.index)
ls_returns_sectors = long_short_returns.loc[common_dates_sectors]
lo_returns_sectors = long_only_returns.loc[common_dates_sectors]
sector_returns_aligned = sector_returns_clean.loc[common_dates_sectors]

# Store results
strategies = ['Long-Short', 'Long-Only']
sector_lfd_results = pd.DataFrame(
    index=strategies,
    columns=['Alpha (annualized)', 'R-squared', 't-stat (alpha)', 'p-value (alpha)']
)

# Store betas for each sector
sector_betas = pd.DataFrame(index=strategies, columns=sector_tickers)

for strategy in strategies:
    if strategy == 'Long-Short':
        y = ls_returns_sectors.values.reshape(-1, 1)
    else:
        y = lo_returns_sectors.values.reshape(-1, 1)
    
    X = sector_returns_aligned.values
    
    # Remove NaN values
    mask = ~(np.isnan(X).any(axis=1) | np.isnan(y).ravel())
    X_clean = X[mask]
    y_clean = y[mask]
    
    # Run multivariate regression
    model = LinearRegression()
    model.fit(X_clean, y_clean)
    
    # Get coefficients
    alpha = model.intercept_[0]
    betas = model.coef_[0]
    
    # Store betas
    sector_betas.loc[strategy] = betas
    
    # Calculate R-squared
    y_pred = model.predict(X_clean)
    ss_res = np.sum((y_clean - y_pred) ** 2)
    ss_tot = np.sum((y_clean - np.mean(y_clean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    # Calculate t-statistic and p-value for alpha
    n = len(y_clean)
    k = X_clean.shape[1]  # number of predictors
    residuals = y_clean - y_pred
    mse = np.sum(residuals ** 2) / (n - k - 1)
    
    # Standard error of alpha (intercept)
    X_with_intercept = np.column_stack([np.ones(n), X_clean])
    XtX_inv = np.linalg.inv(X_with_intercept.T @ X_with_intercept)
    se_alpha = np.sqrt(mse * XtX_inv[0, 0])
    
    t_stat = alpha / se_alpha
    p_value = 2 * (1 - scipy_stats.t.cdf(abs(t_stat), n - k - 1))
    
    # Store results
    sector_lfd_results.loc[strategy, 'Alpha (annualized)'] = alpha * ANNUALIZATION
    sector_lfd_results.loc[strategy, 'R-squared'] = r_squared
    sector_lfd_results.loc[strategy, 't-stat (alpha)'] = t_stat
    sector_lfd_results.loc[strategy, 'p-value (alpha)'] = p_value

print("\n" + "=" * 80)
print("REGRESSION RESULTS: Strategy = alpha + Σ(beta_i * Sector_i) + epsilon")
print("=" * 80)
display(sector_lfd_results.astype(float).style.format({
    'Alpha (annualized)': '{:.4f}',
    'R-squared': '{:.4f}',
    't-stat (alpha)': '{:.2f}',
    'p-value (alpha)': '{:.4f}'
}))

print("\n" + "=" * 80)
print("SECTOR BETAS")
print("=" * 80)
display(sector_betas.astype(float).T.style.format('{:.4f}').background_gradient(cmap='RdYlGn', vmin=-0.5, vmax=0.5))

# Calculate total beta exposure (sum of sector betas)
print("\n" + "=" * 80)
print("TOTAL BETA EXPOSURE")
print("=" * 80)
total_betas = sector_betas.sum(axis=1)
total_beta_df = pd.DataFrame({'Total Beta Exposure': total_betas})
display(total_beta_df.style.format('{:.4f}'))

# Interpretation
print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

for strategy in strategies:
    alpha_ann = sector_lfd_results.loc[strategy, 'Alpha (annualized)']
    r2 = sector_lfd_results.loc[strategy, 'R-squared']
    p_val = sector_lfd_results.loc[strategy, 'p-value (alpha)']
    total_beta = total_betas[strategy]
    
    print(f"\n{strategy}:")
    print(f"  - Alpha: {alpha_ann:.2%} annualized {'(significant)' if p_val < 0.05 else '(not significant at 5%)'}")
    print(f"  - R²: {r2:.2%} (variance explained by sectors)")
    print(f"  - Total Beta: {total_beta:.2f}")
    
    # Find top 3 sector exposures
    strategy_betas = sector_betas.loc[strategy].astype(float)
    top_exposures = strategy_betas.abs().nlargest(3)
    
    print(f"  - Top 3 sector exposures:")
    for sector in top_exposures.index:
        beta_val = strategy_betas[sector]
        print(f"      {sector}: {beta_val:.3f}")

# Comparison to single-factor (SPY) model
print("\n" + "=" * 80)
print("COMPARISON: Single-Factor (SPY) vs Multi-Factor (Sectors)")
print("=" * 80)

comparison = pd.DataFrame({
    'SPY R²': lfd_results['R-squared'].astype(float),
    'Sectors R²': sector_lfd_results['R-squared'].astype(float),
    'Improvement': sector_lfd_results['R-squared'].astype(float) - lfd_results['R-squared'].astype(float)
})

display(comparison.style.format('{:.4f}'))

print("\nConclusion:")
for strategy in strategies:
    improvement = comparison.loc[strategy, 'Improvement']
    if improvement > 0.05:
        print(f"  - {strategy}: Sectors explain significantly more variance than SPY alone")
    elif improvement > 0:
        print(f"  - {strategy}: Sectors explain slightly more variance than SPY alone")
    else:
        print(f"  - {strategy}: SPY explains variance as well as sectors")

MULTIVARIATE LFD - SECTOR EXPOSURE

Using 11 sector ETFs: XLK, XLI, XLF, XLC, XLRE, XLE, XLY, XLB, XLV, XLU, XLP

REGRESSION RESULTS: Strategy = alpha + Σ(beta_i * Sector_i) + epsilon


,Alpha (annualized),R-squared,t-stat (alpha),p-value (alpha)
Long-Short,0.0217,0.6433,0.88,0.3784
Long-Only,0.0266,0.9318,1.41,0.1596



SECTOR BETAS


,Long-Short,Long-Only
XLK,-0.2521,-0.1018
XLI,-0.1367,0.1035
XLF,0.2052,0.2457
XLC,0.0522,0.0644
XLRE,0.2160,0.2569
XLE,0.1108,0.1229
XLY,-0.0965,-0.0100
XLB,0.0046,0.0799
XLV,-0.2438,-0.0673
XLU,0.1429,0.1262



TOTAL BETA EXPOSURE


,Total Beta Exposure
Long-Short,0.0395
Long-Only,0.8142



INTERPRETATION

Long-Short:
  - Alpha: 2.17% annualized (not significant at 5%)
  - R²: 64.33% (variance explained by sectors)
  - Total Beta: 0.04
  - Top 3 sector exposures:
      XLK: -0.252
      XLV: -0.244
      XLRE: 0.216

Long-Only:
  - Alpha: 2.66% annualized (not significant at 5%)
  - R²: 93.18% (variance explained by sectors)
  - Total Beta: 0.81
  - Top 3 sector exposures:
      XLRE: 0.257
      XLF: 0.246
      XLU: 0.126

COMPARISON: Single-Factor (SPY) vs Multi-Factor (Sectors)


,SPY R²,Sectors R²,Improvement
Long-Short,0.0033,0.6433,0.6400
Long-Only,0.6536,0.9318,0.2782



Conclusion:
  - Long-Short: Sectors explain significantly more variance than SPY alone
  - Long-Only: Sectors explain significantly more variance than SPY alone


### 2.3. Sector Neutrality

Is your LO or LS implementation of the carry strategy neutral to sectors? To which sector does it have the largest exposure?

**Remember**

Unless regressors are standardized, comparing betas directly can be misleading. You might consider comparing $\beta_i\sigma_i$ when deciding which exposure is most substantial.

In [15]:
print("=" * 80)
print("SECTOR NEUTRALITY ANALYSIS")
print("=" * 80)

# Calculate volatility of each sector
sector_vols = sector_returns_aligned.std()

# Calculate beta * sigma for each sector (contribution to portfolio vol)
beta_sigma = pd.DataFrame(index=strategies, columns=sector_tickers)

for strategy in strategies:
    for sector in sector_tickers:
        beta = sector_betas.loc[strategy, sector]
        sigma = sector_vols[sector]
        beta_sigma.loc[strategy, sector] = beta * sigma

print("\nβ × σ (Contribution to Portfolio Volatility):")
print("=" * 80)
display(beta_sigma.astype(float).T.style.format('{:.6f}').background_gradient(cmap='RdYlGn'))

# Find largest exposures by beta*sigma
print("\n" + "=" * 80)
print("LARGEST SECTOR EXPOSURES (by β × σ)")
print("=" * 80)

for strategy in strategies:
    strategy_beta_sigma = beta_sigma.loc[strategy].astype(float)
    
    # Top 5 by absolute value
    top_5_abs = strategy_beta_sigma.abs().nlargest(5)
    
    print(f"\n{strategy}:")
    print(f"  Top 5 sectors by |β × σ|:")
    for i, sector in enumerate(top_5_abs.index, 1):
        val = strategy_beta_sigma[sector]
        beta = sector_betas.loc[strategy, sector]
        sigma = sector_vols[sector]
        print(f"    {i}. {sector:6s}: β×σ = {val:7.4f}  (β = {beta:6.3f}, σ = {sigma:.4f})")
    
    # Check if neutral
    total_abs_exposure = strategy_beta_sigma.abs().sum()
    max_abs_exposure = strategy_beta_sigma.abs().max()
    
    print(f"\n  Total |β×σ|: {total_abs_exposure:.4f}")
    print(f"  Max |β×σ|: {max_abs_exposure:.4f}")
    
    if max_abs_exposure < 0.01:
        print(f"  → Strategy appears to be sector-neutral (all exposures < 0.01)")
    else:
        largest_sector = strategy_beta_sigma.abs().idxmax()
        print(f"  → NOT sector-neutral. Largest exposure: {largest_sector}")

# Compare raw betas vs beta*sigma rankings
print("\n" + "=" * 80)
print("COMPARISON: Raw β vs β×σ Rankings")
print("=" * 80)

for strategy in strategies:
    print(f"\n{strategy}:")
    
    raw_betas = sector_betas.loc[strategy].astype(float).abs()
    beta_sigma_vals = beta_sigma.loc[strategy].astype(float).abs()
    
    print(f"  Top 3 by |β|: {', '.join(raw_betas.nlargest(3).index)}")
    print(f"  Top 3 by |β×σ|: {', '.join(beta_sigma_vals.nlargest(3).index)}")
    
    if list(raw_betas.nlargest(3).index) != list(beta_sigma_vals.nlargest(3).index):
        print(f"  ⚠️  Rankings differ! Using β×σ is more appropriate.")
    else:
        print(f"  ✓  Rankings are consistent.")

SECTOR NEUTRALITY ANALYSIS

β × σ (Contribution to Portfolio Volatility):


,Long-Short,Long-Only
XLK,-0.007654,-0.003090
XLI,-0.003881,0.002937
XLF,0.006375,0.007632
XLC,0.001501,0.001853
XLRE,0.006440,0.007663
XLE,0.004550,0.005048
XLY,-0.002886,-0.000298
XLB,0.000132,0.002313
XLV,-0.005682,-0.001568
XLU,0.003814,0.003369



LARGEST SECTOR EXPOSURES (by β × σ)

Long-Short:
  Top 5 sectors by |β × σ|:
    1. XLK   : β×σ = -0.0077  (β = -0.252, σ = 0.0304)
    2. XLRE  : β×σ =  0.0064  (β =  0.216, σ = 0.0298)
    3. XLF   : β×σ =  0.0064  (β =  0.205, σ = 0.0311)
    4. XLV   : β×σ = -0.0057  (β = -0.244, σ = 0.0233)
    5. XLE   : β×σ =  0.0046  (β =  0.111, σ = 0.0411)

  Total |β×σ|: 0.0436
  Max |β×σ|: 0.0077
  → Strategy appears to be sector-neutral (all exposures < 0.01)

Long-Only:
  Top 5 sectors by |β × σ|:
    1. XLRE  : β×σ =  0.0077  (β =  0.257, σ = 0.0298)
    2. XLF   : β×σ =  0.0076  (β =  0.246, σ = 0.0311)
    3. XLE   : β×σ =  0.0050  (β =  0.123, σ = 0.0411)
    4. XLU   : β×σ =  0.0034  (β =  0.126, σ = 0.0267)
    5. XLK   : β×σ = -0.0031  (β = -0.102, σ = 0.0304)

  Total |β×σ|: 0.0359
  Max |β×σ|: 0.0077
  → Strategy appears to be sector-neutral (all exposures < 0.01)

COMPARISON: Raw β vs β×σ Rankings

Long-Short:
  Top 3 by |β|: XLK, XLV, XLRE
  Top 3 by |β×σ|: XLK, XLRE, XLF
  ⚠️

The top 3 absolute betas are 2/3 the same as the top 3 $\beta_i\sigma_i$ values for each sector. The top 3 exposures are to real estate, financials, and energy.

### 2.4. Magnificent Seven

Construct an equally-weighted portfolio of the following tickers. Call this `MAG`. Estimate a LFD on both `SPY` and `MAG`. Do this for both the LO and LS strategies.

Report the stats from `2.1`.

Comment on what you conclude from this regression.

In [16]:
TICKS_MAG = ['AAPL','MSFT','GOOG','AMZN','NVDA','META','TSLA']
display(TICKS_MAG)

['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'META', 'TSLA']

In [17]:
# Magnificent Seven tickers
TICKS_MAG = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'META', 'TSLA']

print("=" * 80)
print("MAGNIFICENT SEVEN PORTFOLIO")
print("=" * 80)
print(f"Tickers: {', '.join(TICKS_MAG)}")

# Extract prices for Magnificent Seven stocks
mag_prices = prices[TICKS_MAG]

# Calculate equally-weighted Magnificent Seven portfolio returns
mag_returns_individual = mag_prices.pct_change()

# Equal weights (1/7 each)
mag_weights = 1.0 / len(TICKS_MAG)
mag_portfolio_returns = mag_returns_individual.mean(axis=1)  # Equal-weighted average

# Align dates
common_dates_mag = long_short_returns.index.intersection(mag_portfolio_returns.index).intersection(spy_returns.index)
ls_returns_mag = long_short_returns.loc[common_dates_mag]
lo_returns_mag = long_only_returns.loc[common_dates_mag]
mag_returns_aligned = mag_portfolio_returns.loc[common_dates_mag]
spy_returns_mag = spy_returns.loc[common_dates_mag]

# Create dataframe with both factors
two_factor_data = pd.DataFrame({
    'Long-Short': ls_returns_mag,
    'Long-Only': lo_returns_mag,
    'SPY': spy_returns_mag,
    'MAG': mag_returns_aligned
})

print(f"\nMAG portfolio statistics (annualized):")
print(f"  Mean: {mag_returns_aligned.mean() * ANNUALIZATION:.2%}")
print(f"  Vol: {mag_returns_aligned.std() * np.sqrt(ANNUALIZATION):.2%}")
print(f"  Sharpe: {(mag_returns_aligned.mean() / mag_returns_aligned.std()) * np.sqrt(ANNUALIZATION):.2f}")

# Two-factor regression: Strategy = alpha + beta_SPY * SPY + beta_MAG * MAG + epsilon
print("\n" + "=" * 80)
print("TWO-FACTOR LFD: Strategy = α + β_SPY × SPY + β_MAG × MAG + ε")
print("=" * 80)

strategies = ['Long-Short', 'Long-Only']
two_factor_results = pd.DataFrame(
    index=strategies,
    columns=['Alpha (ann.)', 'Beta (SPY)', 'Beta (MAG)', 'R-squared', 't-stat (α)', 'p-value (α)']
)

for strategy in strategies:
    y = two_factor_data[strategy].values.reshape(-1, 1)
    X = two_factor_data[['SPY', 'MAG']].values
    
    # Remove NaN values
    mask = ~(np.isnan(X).any(axis=1) | np.isnan(y).ravel())
    X_clean = X[mask]
    y_clean = y[mask]
    
    # Run regression
    model = LinearRegression()
    model.fit(X_clean, y_clean)
    
    # Get coefficients
    alpha = model.intercept_[0]
    beta_spy = model.coef_[0][0]
    beta_mag = model.coef_[0][1]
    
    # Calculate R-squared
    y_pred = model.predict(X_clean)
    ss_res = np.sum((y_clean - y_pred) ** 2)
    ss_tot = np.sum((y_clean - np.mean(y_clean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    # Calculate t-statistic and p-value for alpha
    n = len(y_clean)
    k = 2  # two predictors (SPY and MAG)
    residuals = y_clean - y_pred
    mse = np.sum(residuals ** 2) / (n - k - 1)
    
    # Standard error of alpha
    X_with_intercept = np.column_stack([np.ones(n), X_clean])
    XtX_inv = np.linalg.inv(X_with_intercept.T @ X_with_intercept)
    se_alpha = np.sqrt(mse * XtX_inv[0, 0])
    
    t_stat = alpha / se_alpha
    p_value = 2 * (1 - scipy_stats.t.cdf(abs(t_stat), n - k - 1))
    
    # Store results
    two_factor_results.loc[strategy, 'Alpha (ann.)'] = alpha * ANNUALIZATION
    two_factor_results.loc[strategy, 'Beta (SPY)'] = beta_spy
    two_factor_results.loc[strategy, 'Beta (MAG)'] = beta_mag
    two_factor_results.loc[strategy, 'R-squared'] = r_squared
    two_factor_results.loc[strategy, 't-stat (α)'] = t_stat
    two_factor_results.loc[strategy, 'p-value (α)'] = p_value

display(two_factor_results.astype(float).style.format({
    'Alpha (ann.)': '{:.4f}',
    'Beta (SPY)': '{:.4f}',
    'Beta (MAG)': '{:.4f}',
    'R-squared': '{:.4f}',
    't-stat (α)': '{:.2f}',
    'p-value (α)': '{:.4f}'
}))

# Comparison with single-factor models
print("\n" + "=" * 80)
print("COMPARISON: Single-Factor vs Two-Factor Models")
print("=" * 80)

comparison_models = pd.DataFrame({
    'SPY only R²': lfd_results['R-squared'].astype(float),
    'SPY + MAG R²': two_factor_results['R-squared'].astype(float),
    'Improvement': two_factor_results['R-squared'].astype(float) - lfd_results['R-squared'].astype(float)
})

display(comparison_models.style.format('{:.4f}'))

# Calculate correlation between SPY and MAG
corr_spy_mag = two_factor_data[['SPY', 'MAG']].corr().loc['SPY', 'MAG']
print(f"\nCorrelation between SPY and MAG: {corr_spy_mag:.4f}")

# Interpretation
print("\n" + "=" * 80)
print("INTERPRETATION & CONCLUSIONS")
print("=" * 80)

for strategy in strategies:
    alpha_ann = two_factor_results.loc[strategy, 'Alpha (ann.)']
    beta_spy = two_factor_results.loc[strategy, 'Beta (SPY)']
    beta_mag = two_factor_results.loc[strategy, 'Beta (MAG)']
    r2 = two_factor_results.loc[strategy, 'R-squared']
    p_val = two_factor_results.loc[strategy, 'p-value (α)']
    improvement = comparison_models.loc[strategy, 'Improvement']
    
    print(f"\n{strategy}:")
    print(f"  - Alpha: {alpha_ann:.2%} annualized {'(significant)' if p_val < 0.05 else '(not significant)'}")
    print(f"  - Beta (SPY): {beta_spy:.3f}")
    print(f"  - Beta (MAG): {beta_mag:.3f}")
    print(f"  - R²: {r2:.2%}")
    print(f"  - R² improvement over SPY-only: {improvement:.2%}")
    
    # Interpretation
    if abs(beta_mag) < 0.05:
        print(f"  → Minimal MAG exposure - strategy is not driven by Mag 7 stocks")
    elif beta_mag > 0:
        print(f"  → Positive MAG exposure - strategy tilts toward Mag 7 stocks")
    else:
        print(f"  → Negative MAG exposure - strategy tilts away from Mag 7 stocks")
    
    if improvement > 0.05:
        print(f"  → Adding MAG substantially improves explanatory power")
    elif improvement > 0:
        print(f"  → Adding MAG slightly improves explanatory power")
    else:
        print(f"  → MAG does not add meaningful explanatory power")

MAGNIFICENT SEVEN PORTFOLIO
Tickers: AAPL, MSFT, GOOG, AMZN, NVDA, META, TSLA

MAG portfolio statistics (annualized):
  Mean: 34.98%
  Vol: 26.75%
  Sharpe: 1.31

TWO-FACTOR LFD: Strategy = α + β_SPY × SPY + β_MAG × MAG + ε


,Alpha (ann.),Beta (SPY),Beta (MAG),R-squared,t-stat (α),p-value (α)
Long-Short,0.0464,0.3217,-0.2781,0.1982,1.68,0.0940
Long-Only,0.0671,1.2298,-0.3654,0.7683,2.64,0.0086



COMPARISON: Single-Factor vs Two-Factor Models


,SPY only R²,SPY + MAG R²,Improvement
Long-Short,0.0033,0.1982,0.1949
Long-Only,0.6536,0.7683,0.1147



Correlation between SPY and MAG: 0.8230

INTERPRETATION & CONCLUSIONS

Long-Short:
  - Alpha: 4.64% annualized (not significant)
  - Beta (SPY): 0.322
  - Beta (MAG): -0.278
  - R²: 19.82%
  - R² improvement over SPY-only: 19.49%
  → Negative MAG exposure - strategy tilts away from Mag 7 stocks
  → Adding MAG substantially improves explanatory power

Long-Only:
  - Alpha: 6.71% annualized (significant)
  - Beta (SPY): 1.230
  - Beta (MAG): -0.365
  - R²: 76.83%
  - R² improvement over SPY-only: 11.47%
  → Negative MAG exposure - strategy tilts away from Mag 7 stocks
  → Adding MAG substantially improves explanatory power


***

# 3. Dynamic hedged

It's one thing to ex-post evaluate whether the strategy had alpha beyond sector exposures. That raises the question as to whether we can capture that when we're imperfectly hedging the sector exposure in real time.

At each date we:

1. Look back over a rolling 5-year window of weekly data.
2. Regress the portfolio return on the sector ETF returns.
3. Use the estimated betas as hedge ratios going forward one step.
4. Form a sector-hedged return by combining the strategy with
   an offsetting position in the sector ETFs.

We do this for both the long-only and long-short variants.

### 3.1.

Report the univariate stats of the hedged strategies.

In [18]:
print("=" * 80)
print("DYNAMIC SECTOR-HEDGED STRATEGIES")
print("=" * 80)

# Rolling window parameters
ROLLING_WINDOW = 5 * 52  # 5 years of weekly data

# We need aligned data for all strategies and sectors
common_dates_all = (long_short_returns.index
                    .intersection(long_only_returns.index)
                    .intersection(sector_returns_clean.index))

ls_ret_all = long_short_returns.loc[common_dates_all]
lo_ret_all = long_only_returns.loc[common_dates_all]
sector_ret_all = sector_returns_clean.loc[common_dates_all]

# Initialize storage for hedged returns
ls_hedged_returns = []
lo_hedged_returns = []

print(f"\nRolling window: {ROLLING_WINDOW} weeks (5 years)")
print(f"Total periods: {len(common_dates_all)}")
print(f"Starting hedging from period: {ROLLING_WINDOW + 1}")

# Rolling regression and hedging
for t in range(ROLLING_WINDOW, len(common_dates_all)):
    # Get rolling window data (t-260 to t-1)
    window_start = t - ROLLING_WINDOW
    window_end = t
    
    # === LONG-SHORT STRATEGY ===
    # Historical data for regression
    y_ls_hist = ls_ret_all.iloc[window_start:window_end].values.reshape(-1, 1)
    X_sect_hist = sector_ret_all.iloc[window_start:window_end].values
    
    # Remove NaN values from historical data
    mask_ls = ~(np.isnan(X_sect_hist).any(axis=1) | np.isnan(y_ls_hist).ravel())
    X_ls_clean = X_sect_hist[mask_ls]
    y_ls_clean = y_ls_hist[mask_ls]
    
    # Estimate betas using rolling window
    if len(y_ls_clean) > len(sector_tickers) + 10:  # Need enough data
        model_ls = LinearRegression()
        model_ls.fit(X_ls_clean, y_ls_clean)
        hedge_betas_ls = model_ls.coef_[0]  # Estimated betas
    else:
        hedge_betas_ls = np.zeros(len(sector_tickers))
    
    # Get period t returns (current period)
    strategy_return_ls = ls_ret_all.iloc[t]
    sector_returns_t = sector_ret_all.iloc[t].values
    
    # Calculate hedged return: r_strategy - sum(beta_i * r_sector_i)
    if not np.isnan(strategy_return_ls) and not np.any(np.isnan(sector_returns_t)):
        hedge_return = np.sum(hedge_betas_ls * sector_returns_t)
        hedged_return_ls = strategy_return_ls - hedge_return
    else:
        hedged_return_ls = np.nan
    
    ls_hedged_returns.append(hedged_return_ls)
    
    # === LONG-ONLY STRATEGY ===
    y_lo_hist = lo_ret_all.iloc[window_start:window_end].values.reshape(-1, 1)
    
    # Remove NaN values
    mask_lo = ~(np.isnan(X_sect_hist).any(axis=1) | np.isnan(y_lo_hist).ravel())
    X_lo_clean = X_sect_hist[mask_lo]
    y_lo_clean = y_lo_hist[mask_lo]
    
    # Estimate betas
    if len(y_lo_clean) > len(sector_tickers) + 10:
        model_lo = LinearRegression()
        model_lo.fit(X_lo_clean, y_lo_clean)
        hedge_betas_lo = model_lo.coef_[0]
    else:
        hedge_betas_lo = np.zeros(len(sector_tickers))
    
    # Get period t returns
    strategy_return_lo = lo_ret_all.iloc[t]
    
    # Calculate hedged return
    if not np.isnan(strategy_return_lo) and not np.any(np.isnan(sector_returns_t)):
        hedge_return = np.sum(hedge_betas_lo * sector_returns_t)
        hedged_return_lo = strategy_return_lo - hedge_return
    else:
        hedged_return_lo = np.nan
    
    lo_hedged_returns.append(hedged_return_lo)

# Create series for hedged returns
hedged_dates = common_dates_all[ROLLING_WINDOW:]
ls_hedged_series = pd.Series(ls_hedged_returns, index=hedged_dates)
lo_hedged_series = pd.Series(lo_hedged_returns, index=hedged_dates)

# Combine all strategies for comparison
all_strategies = pd.DataFrame({
    'LS (Unhedged)': ls_ret_all.loc[hedged_dates],
    'LS (Hedged)': ls_hedged_series,
    'LO (Unhedged)': lo_ret_all.loc[hedged_dates],
    'LO (Hedged)': lo_hedged_series
})

print("\n" + "=" * 80)
print("UNIVARIATE STATISTICS - HEDGED VS UNHEDGED")
print("=" * 80)

# Calculate performance metrics
hedged_perf = performance_metrics(all_strategies, annualization=ANNUALIZATION)
display(hedged_perf.style.format('{:.4f}'))

# Calculate tail metrics
print("\n" + "=" * 80)
print("TAIL-RISK STATISTICS - HEDGED VS UNHEDGED")
print("=" * 80)

hedged_tail = tail_metrics(all_strategies, quantile=0.05, relative=False, mdd=True)
display(hedged_tail[['Skewness', 'Kurtosis', 'VaR (0.05)', 'CVaR (0.05)', 'Max Drawdown']].style.format({
    'Skewness': '{:.4f}',
    'Kurtosis': '{:.4f}',
    'VaR (0.05)': '{:.4f}',
    'CVaR (0.05)': '{:.4f}',
    'Max Drawdown': '{:.4f}'
}))

# Comparison
print("\n" + "=" * 80)
print("IMPACT OF HEDGING")
print("=" * 80)

impact_stats = pd.DataFrame({
    'Mean Change': [
        hedged_perf.loc['LS (Hedged)', 'Mean'] - hedged_perf.loc['LS (Unhedged)', 'Mean'],
        hedged_perf.loc['LO (Hedged)', 'Mean'] - hedged_perf.loc['LO (Unhedged)', 'Mean']
    ],
    'Vol Change': [
        hedged_perf.loc['LS (Hedged)', 'Vol'] - hedged_perf.loc['LS (Unhedged)', 'Vol'],
        hedged_perf.loc['LO (Hedged)', 'Vol'] - hedged_perf.loc['LO (Unhedged)', 'Vol']
    ],
    'Sharpe Change': [
        hedged_perf.loc['LS (Hedged)', 'Sharpe'] - hedged_perf.loc['LS (Unhedged)', 'Sharpe'],
        hedged_perf.loc['LO (Hedged)', 'Sharpe'] - hedged_perf.loc['LO (Unhedged)', 'Sharpe']
    ]
}, index=['Long-Short', 'Long-Only'])

display(impact_stats.style.format('{:.4f}').background_gradient(cmap='RdYlGn'))

print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

for strategy in ['Long-Short', 'Long-Only']:
    prefix = 'LS' if strategy == 'Long-Short' else 'LO'
    
    unhedged_mean = hedged_perf.loc[f'{prefix} (Unhedged)', 'Mean']
    hedged_mean = hedged_perf.loc[f'{prefix} (Hedged)', 'Mean']
    unhedged_vol = hedged_perf.loc[f'{prefix} (Unhedged)', 'Vol']
    hedged_vol = hedged_perf.loc[f'{prefix} (Hedged)', 'Vol']
    unhedged_sharpe = hedged_perf.loc[f'{prefix} (Unhedged)', 'Sharpe']
    hedged_sharpe = hedged_perf.loc[f'{prefix} (Hedged)', 'Sharpe']
    
    print(f"\n{strategy}:")
    print(f"  Unhedged: Mean={unhedged_mean:.2%}, Vol={unhedged_vol:.2%}, Sharpe={unhedged_sharpe:.2f}")
    print(f"  Hedged:   Mean={hedged_mean:.2%}, Vol={hedged_vol:.2%}, Sharpe={hedged_sharpe:.2f}")
    
    if hedged_vol < unhedged_vol:
        vol_reduction = (unhedged_vol - hedged_vol) / unhedged_vol
        print(f"  → Volatility reduced by {vol_reduction:.1%}")
    else:
        print(f"  → Volatility increased (hedging added noise)")
    
    if hedged_sharpe > unhedged_sharpe:
        print(f"  → Sharpe ratio improved - successful hedging")
    else:
        print(f"  → Sharpe ratio declined - hedging reduced performance")

print("\nKey Insights:")
print("  - Dynamic hedging removes sector exposure in real-time")
print("  - If hedged returns have lower vol but similar mean → sectors were noise")
print("  - If hedged returns have lower mean → sectors contributed to returns")
print("  - Compare to ex-post alpha from section 2.2 to assess hedge effectiveness")

DYNAMIC SECTOR-HEDGED STRATEGIES

Rolling window: 260 weeks (5 years)
Total periods: 521
Starting hedging from period: 261

UNIVARIATE STATISTICS - HEDGED VS UNHEDGED


,Mean,Vol,Sharpe,Min,Max
LS (Unhedged),0.0127,0.0985,0.1286,-0.0387,0.0739
LS (Hedged),-0.0110,0.0689,-0.1603,-0.0291,0.0350
LO (Unhedged),0.1479,0.1359,1.0878,-0.0724,0.0896
LO (Hedged),0.0028,0.0527,0.0523,-0.0211,0.0231



TAIL-RISK STATISTICS - HEDGED VS UNHEDGED


,Skewness,Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown
LS (Unhedged),0.7136,3.5782,-0.0217,-0.0272,-0.1762
LS (Hedged),0.3273,0.5494,-0.0140,-0.0184,-0.1144
LO (Unhedged),-0.0194,2.3932,-0.0261,-0.0397,-0.1263
LO (Hedged),0.1715,0.3795,-0.0103,-0.0144,-0.0766



IMPACT OF HEDGING


,Mean Change,Vol Change,Sharpe Change
Long-Short,-0.0237,-0.0296,-0.2889
Long-Only,-0.1451,-0.0832,-1.0355



INTERPRETATION

Long-Short:
  Unhedged: Mean=1.27%, Vol=9.85%, Sharpe=0.13
  Hedged:   Mean=-1.10%, Vol=6.89%, Sharpe=-0.16
  → Volatility reduced by 30.1%
  → Sharpe ratio declined - hedging reduced performance

Long-Only:
  Unhedged: Mean=14.79%, Vol=13.59%, Sharpe=1.09
  Hedged:   Mean=0.28%, Vol=5.27%, Sharpe=0.05
  → Volatility reduced by 61.2%
  → Sharpe ratio declined - hedging reduced performance

Key Insights:
  - Dynamic hedging removes sector exposure in real-time
  - If hedged returns have lower vol but similar mean → sectors were noise
  - If hedged returns have lower mean → sectors contributed to returns
  - Compare to ex-post alpha from section 2.2 to assess hedge effectiveness


### 3.2. 

Report sector LFD of the sector-hedged strategies.

In [19]:
print("=" * 80)
print("SECTOR LFD OF DYNAMICALLY HEDGED STRATEGIES")
print("=" * 80)

# Align hedged strategies with sector returns
common_dates_hedged = ls_hedged_series.index.intersection(sector_ret_all.index)
ls_hedged_aligned = ls_hedged_series.loc[common_dates_hedged]
lo_hedged_aligned = lo_hedged_series.loc[common_dates_hedged]
sector_ret_hedged = sector_ret_all.loc[common_dates_hedged]

# Run multivariate regression for hedged strategies
hedged_strategies = {
    'LS (Hedged)': ls_hedged_aligned,
    'LO (Hedged)': lo_hedged_aligned
}

# Store results
hedged_lfd_results = pd.DataFrame(
    index=['LS (Hedged)', 'LO (Hedged)'],
    columns=['Alpha (annualized)', 'R-squared', 't-stat (alpha)', 'p-value (alpha)']
)

# Store betas
hedged_sector_betas = pd.DataFrame(index=['LS (Hedged)', 'LO (Hedged)'], columns=sector_tickers)

for strategy_name, strategy_returns in hedged_strategies.items():
    y = strategy_returns.values.reshape(-1, 1)
    X = sector_ret_hedged.values
    
    # Remove NaN values
    mask = ~(np.isnan(X).any(axis=1) | np.isnan(y).ravel())
    X_clean = X[mask]
    y_clean = y[mask]
    
    # Run regression
    model = LinearRegression()
    model.fit(X_clean, y_clean)
    
    # Get coefficients
    alpha = model.intercept_[0]
    betas = model.coef_[0]
    
    # Store betas
    hedged_sector_betas.loc[strategy_name] = betas
    
    # Calculate R-squared
    y_pred = model.predict(X_clean)
    ss_res = np.sum((y_clean - y_pred) ** 2)
    ss_tot = np.sum((y_clean - np.mean(y_clean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    # Calculate t-statistic and p-value for alpha
    n = len(y_clean)
    k = X_clean.shape[1]
    residuals = y_clean - y_pred
    mse = np.sum(residuals ** 2) / (n - k - 1)
    
    # Standard error of alpha
    X_with_intercept = np.column_stack([np.ones(n), X_clean])
    XtX_inv = np.linalg.inv(X_with_intercept.T @ X_with_intercept)
    se_alpha = np.sqrt(mse * XtX_inv[0, 0])
    
    t_stat = alpha / se_alpha
    p_value = 2 * (1 - scipy_stats.t.cdf(abs(t_stat), n - k - 1))
    
    # Store results
    hedged_lfd_results.loc[strategy_name, 'Alpha (annualized)'] = alpha * ANNUALIZATION
    hedged_lfd_results.loc[strategy_name, 'R-squared'] = r_squared
    hedged_lfd_results.loc[strategy_name, 't-stat (alpha)'] = t_stat
    hedged_lfd_results.loc[strategy_name, 'p-value (alpha)'] = p_value

print("\nRegression Results: Hedged Strategy = alpha + Σ(beta_i * Sector_i) + epsilon")
display(hedged_lfd_results.astype(float).style.format({
    'Alpha (annualized)': '{:.4f}',
    'R-squared': '{:.4f}',
    't-stat (alpha)': '{:.2f}',
    'p-value (alpha)': '{:.4f}'
}))

print("\n" + "=" * 80)
print("SECTOR BETAS (HEDGED STRATEGIES)")
print("=" * 80)
display(hedged_sector_betas.astype(float).T.style.format('{:.4f}').background_gradient(cmap='RdYlGn'))

# Compare unhedged vs hedged sector exposures
print("\n" + "=" * 80)
print("COMPARISON: UNHEDGED vs HEDGED SECTOR EXPOSURES")
print("=" * 80)

# Get R-squared for unhedged strategies (from section 2.2)
# We need to match the indices
unhedged_r2 = pd.Series({
    'LS (Unhedged)': sector_lfd_results.loc['Long-Short', 'R-squared'],
    'LO (Unhedged)': sector_lfd_results.loc['Long-Only', 'R-squared'],
    'LS (Hedged)': hedged_lfd_results.loc['LS (Hedged)', 'R-squared'],
    'LO (Hedged)': hedged_lfd_results.loc['LO (Hedged)', 'R-squared']
})

r2_comparison = pd.DataFrame({
    'Unhedged R²': [
        sector_lfd_results.loc['Long-Short', 'R-squared'],
        sector_lfd_results.loc['Long-Only', 'R-squared']
    ],
    'Hedged R²': [
        hedged_lfd_results.loc['LS (Hedged)', 'R-squared'],
        hedged_lfd_results.loc['LO (Hedged)', 'R-squared']
    ],
    'R² Reduction': [
        float(sector_lfd_results.loc['Long-Short', 'R-squared']) - hedged_lfd_results.loc['LS (Hedged)', 'R-squared'],
        float(sector_lfd_results.loc['Long-Only', 'R-squared']) - hedged_lfd_results.loc['LO (Hedged)', 'R-squared']
    ]
}, index=['Long-Short', 'Long-Only'])

display(r2_comparison.astype(float).style.format('{:.4f}'))

# Compare alpha
alpha_comparison = pd.DataFrame({
    'Unhedged Alpha': [
        sector_lfd_results.loc['Long-Short', 'Alpha (annualized)'],
        sector_lfd_results.loc['Long-Only', 'Alpha (annualized)']
    ],
    'Hedged Alpha': [
        hedged_lfd_results.loc['LS (Hedged)', 'Alpha (annualized)'],
        hedged_lfd_results.loc['LO (Hedged)', 'Alpha (annualized)']
    ],
    'Alpha Change': [
        hedged_lfd_results.loc['LS (Hedged)', 'Alpha (annualized)'] - float(sector_lfd_results.loc['Long-Short', 'Alpha (annualized)']),
        hedged_lfd_results.loc['LO (Hedged)', 'Alpha (annualized)'] - float(sector_lfd_results.loc['Long-Only', 'Alpha (annualized)'])
    ]
}, index=['Long-Short', 'Long-Only'])

print("\n" + "=" * 80)
print("ALPHA COMPARISON")
print("=" * 80)
display(alpha_comparison.astype(float).style.format('{:.4f}'))

# Sum of absolute betas
print("\n" + "=" * 80)
print("TOTAL SECTOR EXPOSURE (Sum of |Beta|)")
print("=" * 80)

total_exposure = pd.DataFrame({
    'Unhedged': [
        sector_betas.loc['Long-Short'].astype(float).abs().sum(),
        sector_betas.loc['Long-Only'].astype(float).abs().sum()
    ],
    'Hedged': [
        hedged_sector_betas.loc['LS (Hedged)'].astype(float).abs().sum(),
        hedged_sector_betas.loc['LO (Hedged)'].astype(float).abs().sum()
    ]
}, index=['Long-Short', 'Long-Only'])

display(total_exposure.style.format('{:.4f}'))

# Interpretation
print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

for i, strategy in enumerate(['Long-Short', 'Long-Only']):
    print(f"\n{strategy}:")
    
    r2_unhedged = r2_comparison.loc[strategy, 'Unhedged R²']
    r2_hedged = r2_comparison.loc[strategy, 'Hedged R²']
    r2_reduction = r2_comparison.loc[strategy, 'R² Reduction']
    
    alpha_unhedged = alpha_comparison.loc[strategy, 'Unhedged Alpha']
    alpha_hedged = alpha_comparison.loc[strategy, 'Hedged Alpha']
    
    print(f"  - Unhedged R²: {r2_unhedged:.2%} → Hedged R²: {r2_hedged:.2%}")
    print(f"  - R² reduction: {r2_reduction:.2%}")
    
    if r2_hedged < 0.05:
        print(f"  → SUCCESS: Hedged strategy is nearly sector-neutral (R² < 5%)")
    elif r2_hedged < r2_unhedged / 2:
        print(f"  → PARTIAL SUCCESS: Sector exposure substantially reduced")
    else:
        print(f"  → LIMITED SUCCESS: Significant sector exposure remains")
    
    print(f"\n  - Unhedged alpha: {alpha_unhedged:.2%} → Hedged alpha: {alpha_hedged:.2%}")
    
    if abs(alpha_hedged) > abs(alpha_unhedged):
        print(f"  → Hedging revealed HIGHER alpha (removed sector drag)")
    elif abs(alpha_hedged) < abs(alpha_unhedged) * 0.5:
        print(f"  → Much of 'alpha' was actually sector exposure")
    else:
        print(f"  → Alpha roughly preserved after hedging")

SECTOR LFD OF DYNAMICALLY HEDGED STRATEGIES

Regression Results: Hedged Strategy = alpha + Σ(beta_i * Sector_i) + epsilon


,Alpha (annualized),R-squared,t-stat (alpha),p-value (alpha)
LS (Hedged),0.0043,0.1826,0.15,0.8827
LO (Hedged),0.0163,0.2445,0.77,0.4447



SECTOR BETAS (HEDGED STRATEGIES)


,LS (Hedged),LO (Hedged)
XLK,0.0425,0.0473
XLI,-0.0284,-0.0117
XLF,-0.0525,-0.0167
XLC,-0.0232,-0.0242
XLRE,0.0596,0.0309
XLE,-0.0059,-0.0286
XLY,-0.0166,-0.0396
XLB,-0.0976,-0.0628
XLV,0.0392,0.0593
XLU,-0.1297,-0.1216



COMPARISON: UNHEDGED vs HEDGED SECTOR EXPOSURES


...


ALPHA COMPARISON


,Unhedged Alpha,Hedged Alpha,Alpha Change
Long-Short,0.0217,0.0043,-0.0175
Long-Only,0.0266,0.0163,-0.0103



TOTAL SECTOR EXPOSURE (Sum of |Beta|)


,Unhedged,Hedged
Long-Short,1.4977,0.6852
Long-Only,1.1848,0.5820



INTERPRETATION

Long-Short:
  - Unhedged R²: 64.33% → Hedged R²: 18.26%
  - R² reduction: 46.07%
  → PARTIAL SUCCESS: Sector exposure substantially reduced

  - Unhedged alpha: 2.17% → Hedged alpha: 0.43%
  → Much of 'alpha' was actually sector exposure

Long-Only:
  - Unhedged R²: 93.18% → Hedged R²: 24.45%
  - R² reduction: 68.72%
  → PARTIAL SUCCESS: Sector exposure substantially reduced

  - Unhedged alpha: 2.66% → Hedged alpha: 1.63%
  → Alpha roughly preserved after hedging


***

# 4. Assessing the Forecast

Up to this point we did **not** build an explicit return forecast;
we jumped straight from a signal (dividend yield) to positions.

We could introduce the intermediate step of the return forecasts to try to understand whether the performance is driven by forecasting vs positioning. For illustration, consider the following forecasts:

- **+0.1%** for stocks we go long (top sorted)
- **−0.1%** for stocks we short (bottom sorted).

### 4.1.
- Treat those implied forecasts as a cross-sectional predictor of
  next-period returns.
- Compute an "out-of-sample" $R^2$ for the forecast vs realized
  returns.
- Compute the correlation between the forecast and realized returns.

In [20]:
print("=" * 80)
print("ASSESSING THE FORECAST - IMPLIED RETURN FORECASTS")
print("=" * 80)

# Forecast values
FORECAST_LONG = 0.001   # +0.1% for top stocks
FORECAST_SHORT = -0.001  # -0.1% for bottom stocks
FORECAST_NEUTRAL = 0.0   # 0% for stocks not in top/bottom 20%

TOP_PCT = 0.20
BOTTOM_PCT = 0.20

print(f"\nImplied forecasts:")
print(f"  - Top 20% (long): {FORECAST_LONG:.2%}")
print(f"  - Bottom 20% (short): {FORECAST_SHORT:.2%}")
print(f"  - Middle 60% (neutral): {FORECAST_NEUTRAL:.2%}")

# Generate forecasts and collect realized returns
all_forecasts = []
all_realized = []
dates_list = []

for t in range(len(dvd_yld.index) - 1):  # -1 because we need next period
    date_t = dvd_yld.index[t]
    date_t1 = dvd_yld.index[t + 1]
    
    # Get dividend yields at time t for ranking
    yields_t = dvd_yld.loc[date_t].dropna()
    
    if len(yields_t) == 0:
        continue
    
    # Calculate how many stocks in top/bottom 20%
    n_stocks = len(yields_t)
    n_top = max(1, int(np.ceil(n_stocks * TOP_PCT)))
    n_bottom = max(1, int(np.ceil(n_stocks * BOTTOM_PCT)))
    
    # Rank stocks
    top_stocks = yields_t.nlargest(n_top).index
    bottom_stocks = yields_t.nsmallest(n_bottom).index
    
    # Create forecast vector for all stocks with yields
    forecasts_t = pd.Series(FORECAST_NEUTRAL, index=yields_t.index)
    forecasts_t[top_stocks] = FORECAST_LONG
    forecasts_t[bottom_stocks] = FORECAST_SHORT
    
    # Get realized returns at t+1
    returns_t1 = returns.loc[date_t1]
    
    # Match stocks that have both forecast and realized return
    common_stocks = forecasts_t.index.intersection(returns_t1.dropna().index)
    
    if len(common_stocks) > 0:
        for stock in common_stocks:
            all_forecasts.append(forecasts_t[stock])
            all_realized.append(returns_t1[stock])
            dates_list.append(date_t1)

# Create arrays
forecasts_array = np.array(all_forecasts)
realized_array = np.array(all_realized)

# Remove any remaining NaN values
valid_mask = ~(np.isnan(forecasts_array) | np.isnan(realized_array))
forecasts_clean = forecasts_array[valid_mask]
realized_clean = realized_array[valid_mask]

print(f"\nTotal stock-period observations: {len(forecasts_clean):,}")

# Calculate out-of-sample R-squared
# R^2 = 1 - SS_res / SS_tot
mean_realized = np.mean(realized_clean)
ss_tot = np.sum((realized_clean - mean_realized) ** 2)
ss_res = np.sum((realized_clean - forecasts_clean) ** 2)
r_squared_oos = 1 - (ss_res / ss_tot)

# Calculate correlation
correlation = np.corrcoef(forecasts_clean, realized_clean)[0, 1]

# Calculate mean absolute error
mae = np.mean(np.abs(realized_clean - forecasts_clean))

# Calculate root mean squared error
rmse = np.sqrt(np.mean((realized_clean - forecasts_clean) ** 2))

print("\n" + "=" * 80)
print("FORECAST EVALUATION METRICS")
print("=" * 80)

metrics_df = pd.DataFrame({
    'Out-of-Sample R²': [r_squared_oos],
    'Correlation': [correlation],
    'MAE': [mae],
    'RMSE': [rmse],
    'Mean Forecast': [np.mean(forecasts_clean)],
    'Mean Realized': [mean_realized]
})

display(metrics_df.T.style.format('{:.6f}'))

# Break down by forecast type
print("\n" + "=" * 80)
print("FORECAST ACCURACY BY GROUP")
print("=" * 80)

forecast_groups = pd.DataFrame({
    'Forecast': forecasts_clean,
    'Realized': realized_clean
})

# Group by forecast value
group_stats = forecast_groups.groupby('Forecast')['Realized'].agg([
    ('Count', 'count'),
    ('Mean Return', 'mean'),
    ('Std Return', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
])

print("\nRealized returns by forecast group:")
display(group_stats.style.format({
    'Count': '{:.0f}',
    'Mean Return': '{:.6f}',
    'Std Return': '{:.6f}',
    'Min': '{:.6f}',
    'Max': '{:.6f}'
}))

# Test if top forecasts actually have higher returns than bottom
if FORECAST_LONG in group_stats.index and FORECAST_SHORT in group_stats.index:
    mean_long = group_stats.loc[FORECAST_LONG, 'Mean Return']
    mean_short = group_stats.loc[FORECAST_SHORT, 'Mean Return']
    spread = mean_long - mean_short
    
    print(f"\nLong-Short Spread:")
    print(f"  Mean return (forecast +0.1%): {mean_long:.4%}")
    print(f"  Mean return (forecast -0.1%): {mean_short:.4%}")
    print(f"  Spread: {spread:.4%}")
    
    # Statistical test
    long_returns = realized_clean[forecasts_clean == FORECAST_LONG]
    short_returns = realized_clean[forecasts_clean == FORECAST_SHORT]
    
    from scipy.stats import ttest_ind
    t_stat, p_value = ttest_ind(long_returns, short_returns)
    
    print(f"  t-statistic: {t_stat:.2f}")
    print(f"  p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print(f"  → Spread is statistically significant at 5% level")
    else:
        print(f"  → Spread is NOT statistically significant")

# Visualization data
print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

print(f"\n1. Out-of-Sample R²: {r_squared_oos:.6f}")
if r_squared_oos < 0:
    print("   → NEGATIVE R²: Forecast worse than just using mean return")
elif r_squared_oos < 0.01:
    print("   → VERY LOW R²: Forecast has minimal predictive power")
elif r_squared_oos < 0.05:
    print("   → LOW R²: Forecast has weak predictive power")
else:
    print(f"   → R² explains {r_squared_oos:.1%} of return variation")

print(f"\n2. Correlation: {correlation:.6f}")
if correlation > 0 and correlation < 0.05:
    print("   → Very weak positive relationship")
elif correlation > 0.05:
    print("   → Positive relationship - forecast directionally correct")
elif correlation < 0:
    print("   → NEGATIVE correlation - forecast is backwards!")

print(f"\n3. Cross-sectional forecasting:")
print("   - This tests if high dividend yield → high returns")
print("   - Low R² is common for equity cross-sectional forecasts")
print("   - Even small predictive power can be profitable when scaled up")

if spread > 0 and p_value < 0.05:
    print("\n4. Strategy viability:")
    print("   → Significant spread indicates dividend yield has predictive power")
    print("   → Long-short strategy can capture this spread")
elif spread > 0 and p_value >= 0.05:
    print("\n4. Strategy viability:")
    print("   → Spread is positive but not statistically significant")
    print("   → Weak evidence for dividend yield as predictor")
else:
    print("\n4. Strategy viability:")
    print("   → No evidence that high dividend yield predicts high returns")

ASSESSING THE FORECAST - IMPLIED RETURN FORECASTS

Implied forecasts:
  - Top 20% (long): 0.10%
  - Bottom 20% (short): -0.10%
  - Middle 60% (neutral): 0.00%

Total stock-period observations: 195,279

FORECAST EVALUATION METRICS


,0
Out-of-Sample R²,-0.004728
Correlation,-0.001089
MAE,0.028853
RMSE,0.041854
Mean Forecast,0.000000
Mean Realized,0.002790



FORECAST ACCURACY BY GROUP

Realized returns by forecast group:


,Count,Mean Return,Std Return,Min,Max
Forecast,,,,,
-0.001000,39292,0.003011,0.042652,-0.610145,0.605402
0.000000,116695,0.002689,0.040282,-0.444261,0.524159
0.001000,39292,0.002868,0.045019,-0.552047,0.705084



Long-Short Spread:
  Mean return (forecast +0.1%): 0.2868%
  Mean return (forecast -0.1%): 0.3011%
  Spread: -0.0143%
  t-statistic: -0.46
  p-value: 0.6468
  → Spread is NOT statistically significant

INTERPRETATION

1. Out-of-Sample R²: -0.004728
   → NEGATIVE R²: Forecast worse than just using mean return

2. Correlation: -0.001089
   → NEGATIVE correlation - forecast is backwards!

3. Cross-sectional forecasting:
   - This tests if high dividend yield → high returns
   - Low R² is common for equity cross-sectional forecasts
   - Even small predictive power can be profitable when scaled up

4. Strategy viability:
   → No evidence that high dividend yield predicts high returns


### 4.2.

In section `3`, we considered a hedged strategies. Repeat `4.1` but this time interpreting our forecasts as pertaining to the hedged residuals of an LFD.

That is, build the forecasts identically, but compare them to the realized $\epsilon_t$ rather than the realized $r_t$. 

Use `SPY` as the single hedging factor. Be careful to estimate $\epsilon_t$ each period with data up to that point in time, rather than using the full sample.


In [21]:
print("=" * 80)
print("4.2 - FORECASTING HEDGED RESIDUALS (SPY HEDGE)")
print("=" * 80)

# We'll estimate epsilon_t by hedging each stock with SPY dynamically
# epsilon_t = r_stock_t - (alpha + beta_SPY * r_SPY_t)

# Get SPY returns aligned with stock returns
spy_ret_aligned = spy_returns.loc[returns.index]

# Rolling window for beta estimation
ROLLING_WINDOW_STOCK = 52  # 1 year for individual stock regressions

# Store forecasts and realized residuals
all_forecasts_residual = []
all_realized_residual = []
dates_list_residual = []

print(f"\nEstimating stock-level residuals using rolling 1-year SPY hedge")
print(f"This may take a moment...")

# For each time period
for t in range(ROLLING_WINDOW_STOCK, len(dvd_yld.index) - 1):
    date_t = dvd_yld.index[t]
    date_t1 = dvd_yld.index[t + 1]
    
    # Get dividend yields at time t for ranking
    yields_t = dvd_yld.loc[date_t].dropna()
    
    if len(yields_t) == 0:
        continue
    
    # Rank stocks and create forecasts (same as before)
    n_stocks = len(yields_t)
    n_top = max(1, int(np.ceil(n_stocks * TOP_PCT)))
    n_bottom = max(1, int(np.ceil(n_stocks * BOTTOM_PCT)))
    
    top_stocks = yields_t.nlargest(n_top).index
    bottom_stocks = yields_t.nsmallest(n_bottom).index
    
    forecasts_t = pd.Series(FORECAST_NEUTRAL, index=yields_t.index)
    forecasts_t[top_stocks] = FORECAST_LONG
    forecasts_t[bottom_stocks] = FORECAST_SHORT
    
    # For each stock, estimate its residual at t+1
    # epsilon_{stock,t+1} = r_{stock,t+1} - (alpha + beta * r_SPY,t+1)
    
    # Get historical window for beta estimation (up to time t)
    window_start = max(0, t - ROLLING_WINDOW_STOCK)
    window_end = t + 1  # Up to and including time t
    
    historical_dates = dvd_yld.index[window_start:window_end]
    
    # Get SPY returns in the window
    spy_hist = spy_ret_aligned.loc[historical_dates].values.reshape(-1, 1)
    
    # Get realized return at t+1 for each stock
    returns_t1 = returns.loc[date_t1]
    spy_t1 = spy_ret_aligned.loc[date_t1]
    
    if pd.isna(spy_t1):
        continue
    
    # Calculate residuals for stocks with forecasts
    common_stocks = forecasts_t.index.intersection(returns_t1.dropna().index)
    
    for stock in common_stocks:
        # Get historical returns for this stock
        stock_hist = returns.loc[historical_dates, stock].values.reshape(-1, 1)
        
        # Remove NaN values from historical data
        mask_hist = ~(np.isnan(spy_hist).ravel() | np.isnan(stock_hist).ravel())
        
        if np.sum(mask_hist) < 20:  # Need sufficient data
            continue
        
        spy_clean = spy_hist[mask_hist]
        stock_clean = stock_hist[mask_hist]
        
        # Estimate beta using historical data (up to time t)
        model_stock = LinearRegression()
        model_stock.fit(spy_clean, stock_clean)
        
        alpha_stock = model_stock.intercept_[0]
        beta_stock = model_stock.coef_[0][0]
        
        # Calculate realized residual at t+1
        r_stock_t1 = returns_t1[stock]
        epsilon_t1 = r_stock_t1 - (alpha_stock + beta_stock * spy_t1)
        
        # Store forecast and realized residual
        all_forecasts_residual.append(forecasts_t[stock])
        all_realized_residual.append(epsilon_t1)
        dates_list_residual.append(date_t1)

# Create arrays
forecasts_residual_array = np.array(all_forecasts_residual)
realized_residual_array = np.array(all_realized_residual)

# Remove any remaining NaN values
valid_mask_res = ~(np.isnan(forecasts_residual_array) | np.isnan(realized_residual_array))
forecasts_res_clean = forecasts_residual_array[valid_mask_res]
realized_res_clean = realized_residual_array[valid_mask_res]

print(f"Total stock-period observations: {len(forecasts_res_clean):,}")

# Calculate metrics for residual forecasts
mean_realized_res = np.mean(realized_res_clean)
ss_tot_res = np.sum((realized_res_clean - mean_realized_res) ** 2)
ss_res_res = np.sum((realized_res_clean - forecasts_res_clean) ** 2)
r_squared_oos_res = 1 - (ss_res_res / ss_tot_res)

correlation_res = np.corrcoef(forecasts_res_clean, realized_res_clean)[0, 1]
mae_res = np.mean(np.abs(realized_res_clean - forecasts_res_clean))
rmse_res = np.sqrt(np.mean((realized_res_clean - forecasts_res_clean) ** 2))

print("\n" + "=" * 80)
print("FORECAST EVALUATION - HEDGED RESIDUALS (SPY)")
print("=" * 80)

metrics_residual_df = pd.DataFrame({
    'Out-of-Sample R²': [r_squared_oos_res],
    'Correlation': [correlation_res],
    'MAE': [mae_res],
    'RMSE': [rmse_res],
    'Mean Forecast': [np.mean(forecasts_res_clean)],
    'Mean Realized ε': [mean_realized_res]
})

display(metrics_residual_df.T.style.format('{:.6f}'))

# Break down by forecast type
print("\n" + "=" * 80)
print("REALIZED RESIDUALS BY FORECAST GROUP")
print("=" * 80)

forecast_groups_res = pd.DataFrame({
    'Forecast': forecasts_res_clean,
    'Realized ε': realized_res_clean
})

group_stats_res = forecast_groups_res.groupby('Forecast')['Realized ε'].agg([
    ('Count', 'count'),
    ('Mean ε', 'mean'),
    ('Std ε', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
])

display(group_stats_res.style.format({
    'Count': '{:.0f}',
    'Mean ε': '{:.6f}',
    'Std ε': '{:.6f}',
    'Min': '{:.6f}',
    'Max': '{:.6f}'
}))

# Test spread in residuals
if FORECAST_LONG in group_stats_res.index and FORECAST_SHORT in group_stats_res.index:
    mean_long_res = group_stats_res.loc[FORECAST_LONG, 'Mean ε']
    mean_short_res = group_stats_res.loc[FORECAST_SHORT, 'Mean ε']
    spread_res = mean_long_res - mean_short_res
    
    print(f"\nLong-Short Spread in Residuals:")
    print(f"  Mean ε (forecast +0.1%): {mean_long_res:.4%}")
    print(f"  Mean ε (forecast -0.1%): {mean_short_res:.4%}")
    print(f"  Spread: {spread_res:.4%}")
    
    # Statistical test
    long_residuals = realized_res_clean[forecasts_res_clean == FORECAST_LONG]
    short_residuals = realized_res_clean[forecasts_res_clean == FORECAST_SHORT]
    
    t_stat_res, p_value_res = ttest_ind(long_residuals, short_residuals)
    
    print(f"  t-statistic: {t_stat_res:.2f}")
    print(f"  p-value: {p_value_res:.4f}")
    
    if p_value_res < 0.05:
        print(f"  → Spread in residuals is statistically significant")
    else:
        print(f"  → Spread in residuals is NOT statistically significant")

# Compare to raw returns (section 4.1)
print("\n" + "=" * 80)
print("COMPARISON: RAW RETURNS vs SPY-HEDGED RESIDUALS")
print("=" * 80)

comparison_4_1_4_2 = pd.DataFrame({
    'Raw Returns (4.1)': [r_squared_oos, correlation, spread if 'spread' in locals() else np.nan],
    'SPY-Hedged (4.2)': [r_squared_oos_res, correlation_res, spread_res if 'spread_res' in locals() else np.nan]
}, index=['Out-of-Sample R²', 'Correlation', 'Long-Short Spread'])

display(comparison_4_1_4_2.style.format('{:.6f}'))

print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

print("\n1. Does dividend yield forecast stock-specific returns (beyond market)?")
if r_squared_oos_res > r_squared_oos:
    print("   → R² INCREASED after hedging - dividend yield predicts idiosyncratic returns")
elif r_squared_oos_res < r_squared_oos / 2:
    print("   → R² DECREASED significantly - dividend yield mainly forecasts market exposure")
else:
    print("   → R² similar - dividend yield forecasts both market and idiosyncratic components")

print("\n2. Is the predictive power stock-specific or market-driven?")
if correlation_res > correlation * 0.8:
    print("   → Correlation preserved - predictive power is stock-specific")
else:
    print("   → Correlation reduced - much of predictive power was market exposure")

print("\n3. Does the long-short spread persist after market hedging?")
if 'spread_res' in locals() and 'spread' in locals():
    if spread_res > spread * 0.8:
        print("   → Spread preserved - strategy captures stock selection alpha")
    else:
        print("   → Spread reduced - strategy returns partly driven by market beta")

4.2 - FORECASTING HEDGED RESIDUALS (SPY HEDGE)

Estimating stock-level residuals using rolling 1-year SPY hedge
This may take a moment...
Total stock-period observations: 176,382

FORECAST EVALUATION - HEDGED RESIDUALS (SPY)


,0
Out-of-Sample R²,0.000384
Correlation,0.024103
MAE,0.023946
RMSE,0.034369
Mean Forecast,-0.000000
Mean Realized ε,-0.000441



REALIZED RESIDUALS BY FORECAST GROUP


,Count,Mean ε,Std ε,Min,Max
Forecast,,,,,
-0.001000,35478,-0.001565,0.034829,-0.462492,0.541149
0.000000,105426,-0.000564,0.033281,-0.324303,0.546577
0.001000,35478,0.001048,0.036973,-0.418239,0.479519



Long-Short Spread in Residuals:
  Mean ε (forecast +0.1%): 0.1048%
  Mean ε (forecast -0.1%): -0.1565%
  Spread: 0.2613%
  t-statistic: 9.69
  p-value: 0.0000
  → Spread in residuals is statistically significant

COMPARISON: RAW RETURNS vs SPY-HEDGED RESIDUALS


,Raw Returns (4.1),SPY-Hedged (4.2)
Out-of-Sample R²,-0.004728,0.000384
Correlation,-0.001089,0.024103
Long-Short Spread,-0.000143,0.002613



INTERPRETATION

1. Does dividend yield forecast stock-specific returns (beyond market)?
   → R² INCREASED after hedging - dividend yield predicts idiosyncratic returns

2. Is the predictive power stock-specific or market-driven?
   → Correlation preserved - predictive power is stock-specific

3. Does the long-short spread persist after market hedging?
   → Spread preserved - strategy captures stock selection alpha


### 4.3.

Repeat `4.2.` but considering all the sectors as the LFD.

In [22]:
print("=" * 80)
print("4.3 - FORECASTING HEDGED RESIDUALS (SECTOR HEDGE)")
print("=" * 80)

# Now we'll estimate epsilon_t by hedging each stock with ALL sector ETFs
# epsilon_t = r_stock_t - (alpha + sum(beta_i * r_sector_i,t))

# Get sector returns aligned with stock returns
sector_ret_stocks = sector_returns_clean.loc[returns.index]

# Rolling window for beta estimation
ROLLING_WINDOW_SECTOR = 52  # 1 year for individual stock regressions

# Store forecasts and realized residuals
all_forecasts_sector_res = []
all_realized_sector_res = []
dates_list_sector_res = []

print(f"\nEstimating stock-level residuals using rolling 1-year sector hedge")
print(f"Using {len(sector_tickers)} sector ETFs: {', '.join(sector_tickers[:5])}...")
print(f"This may take a few moments...")

# For each time period
for t in range(ROLLING_WINDOW_SECTOR, len(dvd_yld.index) - 1):
    if t % 50 == 0:  # Progress indicator
        print(f"  Processing period {t}/{len(dvd_yld.index)}...")
    
    date_t = dvd_yld.index[t]
    date_t1 = dvd_yld.index[t + 1]
    
    # Get dividend yields at time t for ranking
    yields_t = dvd_yld.loc[date_t].dropna()
    
    if len(yields_t) == 0:
        continue
    
    # Rank stocks and create forecasts (same as before)
    n_stocks = len(yields_t)
    n_top = max(1, int(np.ceil(n_stocks * TOP_PCT)))
    n_bottom = max(1, int(np.ceil(n_stocks * BOTTOM_PCT)))
    
    top_stocks = yields_t.nlargest(n_top).index
    bottom_stocks = yields_t.nsmallest(n_bottom).index
    
    forecasts_t = pd.Series(FORECAST_NEUTRAL, index=yields_t.index)
    forecasts_t[top_stocks] = FORECAST_LONG
    forecasts_t[bottom_stocks] = FORECAST_SHORT
    
    # Get historical window for beta estimation (up to time t)
    window_start = max(0, t - ROLLING_WINDOW_SECTOR)
    window_end = t + 1  # Up to and including time t
    
    historical_dates = dvd_yld.index[window_start:window_end]
    
    # Get sector returns in the window
    sectors_hist = sector_ret_stocks.loc[historical_dates].values
    
    # Get realized return at t+1 for each stock and sectors
    returns_t1 = returns.loc[date_t1]
    sectors_t1 = sector_ret_stocks.loc[date_t1].values
    
    if np.any(np.isnan(sectors_t1)):
        continue
    
    # Calculate residuals for stocks with forecasts
    common_stocks = forecasts_t.index.intersection(returns_t1.dropna().index)
    
    for stock in common_stocks:
        # Get historical returns for this stock
        stock_hist = returns.loc[historical_dates, stock].values.reshape(-1, 1)
        
        # Remove NaN values from historical data
        mask_hist = ~(np.isnan(sectors_hist).any(axis=1) | np.isnan(stock_hist).ravel())
        
        if np.sum(mask_hist) < 30:  # Need sufficient data (more than # of sectors)
            continue
        
        sectors_clean = sectors_hist[mask_hist]
        stock_clean = stock_hist[mask_hist]
        
        # Estimate betas using historical data (up to time t)
        try:
            model_stock_sector = LinearRegression()
            model_stock_sector.fit(sectors_clean, stock_clean)
            
            alpha_stock = model_stock_sector.intercept_[0]
            betas_stock = model_stock_sector.coef_[0]
            
            # Calculate realized residual at t+1
            r_stock_t1 = returns_t1[stock]
            predicted_return = alpha_stock + np.sum(betas_stock * sectors_t1)
            epsilon_sector_t1 = r_stock_t1 - predicted_return
            
            # Store forecast and realized residual
            all_forecasts_sector_res.append(forecasts_t[stock])
            all_realized_sector_res.append(epsilon_sector_t1)
            dates_list_sector_res.append(date_t1)
        except:
            # Skip if regression fails (e.g., singular matrix)
            continue

print("Done!")

# Create arrays
forecasts_sector_array = np.array(all_forecasts_sector_res)
realized_sector_array = np.array(all_realized_sector_res)

# Remove any remaining NaN values
valid_mask_sector = ~(np.isnan(forecasts_sector_array) | np.isnan(realized_sector_array))
forecasts_sector_clean = forecasts_sector_array[valid_mask_sector]
realized_sector_clean = realized_sector_array[valid_mask_sector]

print(f"\nTotal stock-period observations: {len(forecasts_sector_clean):,}")

# Calculate metrics for sector-hedged residual forecasts
mean_realized_sector = np.mean(realized_sector_clean)
ss_tot_sector = np.sum((realized_sector_clean - mean_realized_sector) ** 2)
ss_res_sector = np.sum((realized_sector_clean - forecasts_sector_clean) ** 2)
r_squared_oos_sector = 1 - (ss_res_sector / ss_tot_sector)

correlation_sector = np.corrcoef(forecasts_sector_clean, realized_sector_clean)[0, 1]
mae_sector = np.mean(np.abs(realized_sector_clean - forecasts_sector_clean))
rmse_sector = np.sqrt(np.mean((realized_sector_clean - forecasts_sector_clean) ** 2))

print("\n" + "=" * 80)
print("FORECAST EVALUATION - HEDGED RESIDUALS (SECTORS)")
print("=" * 80)

metrics_sector_df = pd.DataFrame({
    'Out-of-Sample R²': [r_squared_oos_sector],
    'Correlation': [correlation_sector],
    'MAE': [mae_sector],
    'RMSE': [rmse_sector],
    'Mean Forecast': [np.mean(forecasts_sector_clean)],
    'Mean Realized ε': [mean_realized_sector]
})

display(metrics_sector_df.T.style.format('{:.6f}'))

# Break down by forecast type
print("\n" + "=" * 80)
print("REALIZED RESIDUALS BY FORECAST GROUP")
print("=" * 80)

forecast_groups_sector = pd.DataFrame({
    'Forecast': forecasts_sector_clean,
    'Realized ε': realized_sector_clean
})

group_stats_sector = forecast_groups_sector.groupby('Forecast')['Realized ε'].agg([
    ('Count', 'count'),
    ('Mean ε', 'mean'),
    ('Std ε', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
])

display(group_stats_sector.style.format({
    'Count': '{:.0f}',
    'Mean ε': '{:.6f}',
    'Std ε': '{:.6f}',
    'Min': '{:.6f}',
    'Max': '{:.6f}'
}))

# Test spread in sector-hedged residuals
if FORECAST_LONG in group_stats_sector.index and FORECAST_SHORT in group_stats_sector.index:
    mean_long_sector = group_stats_sector.loc[FORECAST_LONG, 'Mean ε']
    mean_short_sector = group_stats_sector.loc[FORECAST_SHORT, 'Mean ε']
    spread_sector = mean_long_sector - mean_short_sector
    
    print(f"\nLong-Short Spread in Sector-Hedged Residuals:")
    print(f"  Mean ε (forecast +0.1%): {mean_long_sector:.4%}")
    print(f"  Mean ε (forecast -0.1%): {mean_short_sector:.4%}")
    print(f"  Spread: {spread_sector:.4%}")
    
    # Statistical test
    long_sector_residuals = realized_sector_clean[forecasts_sector_clean == FORECAST_LONG]
    short_sector_residuals = realized_sector_clean[forecasts_sector_clean == FORECAST_SHORT]
    
    t_stat_sector, p_value_sector = ttest_ind(long_sector_residuals, short_sector_residuals)
    
    print(f"  t-statistic: {t_stat_sector:.2f}")
    print(f"  p-value: {p_value_sector:.4f}")
    
    if p_value_sector < 0.05:
        print(f"  → Spread in sector-hedged residuals is statistically significant")
    else:
        print(f"  → Spread is NOT statistically significant")

# Compare all three versions
print("\n" + "=" * 80)
print("COMPARISON: RAW vs SPY-HEDGED vs SECTOR-HEDGED")
print("=" * 80)

comparison_all = pd.DataFrame({
    'Raw Returns (4.1)': [
        r_squared_oos, 
        correlation, 
        spread if 'spread' in locals() else np.nan,
        p_value if 'p_value' in locals() else np.nan
    ],
    'SPY-Hedged (4.2)': [
        r_squared_oos_res, 
        correlation_res, 
        spread_res if 'spread_res' in locals() else np.nan,
        p_value_res if 'p_value_res' in locals() else np.nan
    ],
    'Sector-Hedged (4.3)': [
        r_squared_oos_sector, 
        correlation_sector, 
        spread_sector if 'spread_sector' in locals() else np.nan,
        p_value_sector if 'p_value_sector' in locals() else np.nan
    ]
}, index=['Out-of-Sample R²', 'Correlation', 'Long-Short Spread', 'p-value'])

display(comparison_all.style.format('{:.6f}'))

print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)

print("\n1. Progression of R² across hedge levels:")
print(f"   Raw: {r_squared_oos:.6f} → SPY: {r_squared_oos_res:.6f} → Sectors: {r_squared_oos_sector:.6f}")

if r_squared_oos_sector > r_squared_oos_res > r_squared_oos:
    print("   → R² INCREASES with hedging - dividend yield forecasts idiosyncratic returns!")
elif r_squared_oos_sector > r_squared_oos:
    print("   → R² higher with sector hedge - dividend yield forecasts stock-specific component")
elif r_squared_oos_sector < r_squared_oos / 2:
    print("   → R² COLLAPSES with sector hedge - dividend yield mainly forecasts factor exposure")
else:
    print("   → Mixed results - some idiosyncratic forecasting power")

print("\n2. Forecast correlations:")
print(f"   Raw: {correlation:.6f} → SPY: {correlation_res:.6f} → Sectors: {correlation_sector:.6f}")

if correlation_sector > correlation * 0.8:
    print("   → Correlation preserved - strong stock-selection signal")
elif correlation_sector > 0 and correlation_sector < correlation * 0.5:
    print("   → Correlation weakened - some factor exposure in signal")
else:
    print("   → Correlation degraded - signal was mostly factor-driven")

4.3 - FORECASTING HEDGED RESIDUALS (SECTOR HEDGE)

Estimating stock-level residuals using rolling 1-year sector hedge
Using 11 sector ETFs: XLK, XLI, XLF, XLC, XLRE...
This may take a few moments...
  Processing period 100/522...
  Processing period 150/522...
  Processing period 200/522...
  Processing period 250/522...
  Processing period 300/522...
  Processing period 350/522...
  Processing period 400/522...
  Processing period 450/522...
  Processing period 500/522...
Done!

Total stock-period observations: 127,311

FORECAST EVALUATION - HEDGED RESIDUALS (SECTORS)


,0
Out-of-Sample R²,0.000201
Correlation,0.015645
MAE,0.023874
RMSE,0.034272
Mean Forecast,-0.000000
Mean Realized ε,-0.000204



REALIZED RESIDUALS BY FORECAST GROUP


,Count,Mean ε,Std ε,Min,Max
Forecast,,,,,
-0.001000,25644,-0.001136,0.036708,-0.515124,0.355864
0.000000,76070,-0.000145,0.033735,-0.343738,0.716576
0.001000,25597,0.000555,0.033311,-0.372926,0.386233



Long-Short Spread in Sector-Hedged Residuals:
  Mean ε (forecast +0.1%): 0.0555%
  Mean ε (forecast -0.1%): -0.1136%
  Spread: 0.1690%
  t-statistic: 5.46
  p-value: 0.0000
  → Spread in sector-hedged residuals is statistically significant

COMPARISON: RAW vs SPY-HEDGED vs SECTOR-HEDGED


,Raw Returns (4.1),SPY-Hedged (4.2),Sector-Hedged (4.3)
Out-of-Sample R²,-0.004728,0.000384,0.000201
Correlation,-0.001089,0.024103,0.015645
Long-Short Spread,-0.000143,0.002613,0.001690
p-value,0.646811,0.000000,0.000000



INTERPRETATION

1. Progression of R² across hedge levels:
   Raw: -0.004728 → SPY: 0.000384 → Sectors: 0.000201
   → R² higher with sector hedge - dividend yield forecasts stock-specific component

2. Forecast correlations:
   Raw: -0.001089 → SPY: 0.024103 → Sectors: 0.015645
   → Correlation preserved - strong stock-selection signal


***

# 5. No-Arbitrage via Cross-Asset Replication

If an asset can be replicated by other assets, an arbitrage-free model should assign consistent forecasts to the asset and its replication portfolio.

### 5.1. LASSO Replication and Forecast Consistency

For a selected asset, use LASSO to find a replication portfolio. Compare the dividend-yield-based forecast for the target asset vs its replication portfolio. 

In [23]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# Select target asset - let's choose AAPL (high dividend yield, well-known)
TARGET_ASSET = 'AAPL'

print(f"\nTarget Asset: {TARGET_ASSET}")

# Verify target asset has sufficient data
if TARGET_ASSET not in prices.columns:
    print(f"Error: {TARGET_ASSET} not in dataset")
else:
    target_prices = prices[TARGET_ASSET]
    target_returns = target_prices.pct_change()
    target_yields = dvd_yld[TARGET_ASSET]
    
    print(f"Data available: {target_returns.notna().sum()} periods")
    print(f"Mean return: {target_returns.mean() * 52:.2%} annualized")
    print(f"Volatility: {target_returns.std() * np.sqrt(52):.2%} annualized")
    print(f"Mean dividend yield: {target_yields.mean():.2%}")

# Use all other stocks as potential replicators
# Exclude the target asset
replicator_stocks = [col for col in prices.columns if col != TARGET_ASSET]

print(f"\nUsing {len(replicator_stocks)} stocks as potential replicators")

# Rolling LASSO replication
ROLLING_WINDOW_LASSO = 2 * 52  # 2 years for LASSO estimation

# Store replication results
lasso_weights_history = []
replication_errors = []
target_forecasts = []
replication_forecasts = []
dates_lasso = []

print(f"\nPerforming rolling LASSO replication (2-year window)...")

for t in range(ROLLING_WINDOW_LASSO, len(prices.index) - 1):
    if t % 100 == 0:
        print(f"  Period {t}/{len(prices.index)}...")
    
    date_t = prices.index[t]
    date_t1 = prices.index[t + 1]
    
    # Historical window
    window_start = t - ROLLING_WINDOW_LASSO
    window_end = t + 1
    
    hist_dates = prices.index[window_start:window_end]
    
    # Get target returns in historical window
    y_target = target_returns.loc[hist_dates].values
    
    # Get replicator returns in historical window
    X_replicators = prices[replicator_stocks].loc[hist_dates].pct_change().values
    
    # Remove NaN values
    mask = ~(np.isnan(y_target) | np.isnan(X_replicators).any(axis=1))
    
    if np.sum(mask) < ROLLING_WINDOW_LASSO * 0.8:  # Need most of the data
        continue
    
    y_clean = y_target[mask]
    X_clean = X_replicators[mask]
    
    # Standardize features for LASSO
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_clean)
    
    # Fit LASSO with cross-validation to choose alpha
    try:
        lasso = LassoCV(cv=5, max_iter=2000, random_state=42, n_jobs=-1)
        lasso.fit(X_scaled, y_clean)
        
        # Get LASSO weights
        lasso_weights = lasso.coef_
        
        # Count non-zero weights (sparse portfolio)
        n_nonzero = np.sum(np.abs(lasso_weights) > 1e-6)
        
        # Get returns at t+1 for replication
        target_return_t1 = target_returns.loc[date_t1]
        replicator_returns_t1 = prices[replicator_stocks].pct_change().loc[date_t1].values
        
        # Calculate replication portfolio return
        if not np.isnan(target_return_t1) and not np.any(np.isnan(replicator_returns_t1)):
            # Transform current returns with same scaling
            X_t1_scaled = scaler.transform(replicator_returns_t1.reshape(1, -1))
            replication_return = lasso.predict(X_t1_scaled)[0]
            
            replication_error = target_return_t1 - replication_return
            
            # Get dividend yield forecasts
            target_yield_t = target_yields.loc[date_t] if date_t in target_yields.index else np.nan
            
            # Calculate replication portfolio's implied dividend yield
            # Weight-average the dividend yields of stocks in the replication portfolio
            rep_yields_t = dvd_yld[replicator_stocks].loc[date_t].values
            
            # For stocks with non-zero LASSO weights, get their dividend yields
            # Use absolute weights to calculate weighted average
            abs_weights = np.abs(lasso_weights)
            if abs_weights.sum() > 1e-6:
                valid_yields_mask = ~np.isnan(rep_yields_t)
                if valid_yields_mask.any():
                    weights_valid = abs_weights[valid_yields_mask]
                    yields_valid = rep_yields_t[valid_yields_mask]
                    
                    if weights_valid.sum() > 0:
                        replication_yield = np.average(yields_valid, weights=weights_valid)
                    else:
                        replication_yield = np.nan
                else:
                    replication_yield = np.nan
            else:
                replication_yield = np.nan
            
            # Store results
            lasso_weights_history.append(lasso_weights)
            replication_errors.append(replication_error)
            target_forecasts.append(target_yield_t)
            replication_forecasts.append(replication_yield)
            dates_lasso.append(date_t)
    
    except Exception as e:
        continue

# Check if we have any data
if len(replication_errors) == 0:
    print("\nERROR: No valid replication periods found. Try a different asset or shorter window.")
else:
    # Analyze replication quality
    replication_errors_array = np.array(replication_errors)
    tracking_error = np.std(replication_errors_array) * np.sqrt(52)
    
    print("\n" + "=" * 80)
    print("REPLICATION QUALITY")
    print("=" * 80)
    
    print(f"\nTracking Error (annualized): {tracking_error:.2%}")
    print(f"Mean Replication Error: {np.mean(replication_errors_array):.4%}")
    print(f"RMSE: {np.sqrt(np.mean(replication_errors_array**2)):.4%}")
    
    # Analyze forecast consistency
    target_fcst_array = np.array(target_forecasts)
    replication_fcst_array = np.array(replication_forecasts)
    
    # Remove NaN values
    valid_fcst_mask = ~(np.isnan(target_fcst_array) | np.isnan(replication_fcst_array))
    target_fcst_clean = target_fcst_array[valid_fcst_mask]
    replication_fcst_clean = replication_fcst_array[valid_fcst_mask]
    
    if len(target_fcst_clean) == 0:
        print("\nWARNING: No valid forecast pairs found (dividend yield data may be missing)")
    else:
        print("\n" + "=" * 80)
        print("FORECAST CONSISTENCY")
        print("=" * 80)
        
        print(f"\nValid forecast pairs: {len(target_fcst_clean)}")
        print(f"\nTarget asset ({TARGET_ASSET}) dividend yield:")
        print(f"  Mean: {np.mean(target_fcst_clean):.2%}")
        print(f"  Std: {np.std(target_fcst_clean):.2%}")
        
        print(f"\nReplication portfolio dividend yield:")
        print(f"  Mean: {np.mean(replication_fcst_clean):.2%}")
        print(f"  Std: {np.std(replication_fcst_clean):.2%}")
        
        # Calculate forecast differences
        forecast_diff = target_fcst_clean - replication_fcst_clean
        print(f"\nForecast Differences:")
        print(f"  Mean difference: {np.mean(forecast_diff):.2%}")
        print(f"  Std of difference: {np.std(forecast_diff):.2%}")
        
        if len(forecast_diff) > 0:
            print(f"  Max absolute difference: {np.max(np.abs(forecast_diff)):.2%}")
        
        # Correlation between forecasts
        corr_forecasts = np.corrcoef(target_fcst_clean, replication_fcst_clean)[0, 1]
        print(f"\nCorrelation between forecasts: {corr_forecasts:.4f}")
        
        # Regression: replication forecast on target forecast
        from scipy.stats import linregress
        slope, intercept, r_value, p_value_reg, std_err = linregress(target_fcst_clean, replication_fcst_clean)
        
        print(f"\nRegression: Replication Yield = α + β × Target Yield")
        print(f"  Intercept (α): {intercept:.4f}")
        print(f"  Slope (β): {slope:.4f}  [should be ≈1 for consistency]")
        print(f"  R²: {r_value**2:.4f}")
        
        # Test for forecast consistency
        print("\n" + "=" * 80)
        print("NO-ARBITRAGE IMPLICATIONS")
        print("=" * 80)
        
        print("\n1. Replication Quality:")
        if tracking_error < 0.05:
            print(f"   → Excellent replication (TE = {tracking_error:.2%})")
            print(f"   → Assets can be effectively replicated with sparse portfolio")
        elif tracking_error < 0.10:
            print(f"   → Good replication (TE = {tracking_error:.2%})")
        else:
            print(f"   → Poor replication (TE = {tracking_error:.2%})")
            print(f"   → Target asset has unique characteristics")
        
        print("\n2. Forecast Consistency:")
        if abs(slope - 1.0) < 0.2 and abs(intercept) < 0.01:
            print(f"   → Forecasts are CONSISTENT (β ≈ 1, α ≈ 0)")
            print(f"   → No arbitrage condition approximately holds")
        elif abs(np.mean(forecast_diff)) > 0.01:
            print(f"   → Forecasts are INCONSISTENT")
            print(f"   → Mean difference: {np.mean(forecast_diff):.2%}")
            print(f"   → Potential arbitrage opportunity!")
        else:
            print(f"   → Forecasts are somewhat consistent")
        
        print("\n3. Arbitrage Strategy:")
        if abs(np.mean(forecast_diff)) > 0.01:
            if np.mean(forecast_diff) > 0:
                print(f"   → Target asset has HIGHER forecast yield")
                print(f"   → Potential: LONG {TARGET_ASSET}, SHORT replication portfolio")
            else:
                print(f"   → Replication portfolio has HIGHER forecast yield")
                print(f"   → Potential: SHORT {TARGET_ASSET}, LONG replication portfolio")
            print(f"   → Expected yield pickup: {abs(np.mean(forecast_diff)):.2%}")
        else:
            print(f"   → No significant forecast discrepancy")
            print(f"   → No arbitrage opportunity based on dividend yields")
    
    # Show example of LASSO weights (last period)
    if len(lasso_weights_history) > 0:
        print("\n" + "=" * 80)
        print("EXAMPLE: LASSO REPLICATION PORTFOLIO (LAST PERIOD)")
        print("=" * 80)
        
        last_weights = lasso_weights_history[-1]
        nonzero_idx = np.abs(last_weights) > 1e-6
        
        print(f"\nNumber of stocks in replication: {np.sum(nonzero_idx)}")
        print(f"Top 10 holdings:")
        
        weights_df = pd.DataFrame({
            'Stock': np.array(replicator_stocks)[nonzero_idx],
            'Weight': last_weights[nonzero_idx]
        }).sort_values('Weight', key=abs, ascending=False).head(10)
        
        display(weights_df.style.format({'Weight': '{:.4f}'}))


Target Asset: AAPL
Data available: 521 periods
Mean return: 23.51% annualized
Volatility: 27.91% annualized
Mean dividend yield: 116.71%

Using 484 stocks as potential replicators

Performing rolling LASSO replication (2-year window)...
  Period 200/522...
  Period 300/522...
  Period 400/522...
  Period 500/522...

REPLICATION QUALITY

Tracking Error (annualized): 20.70%
Mean Replication Error: -0.1488%
RMSE: 2.8751%

FORECAST CONSISTENCY

Valid forecast pairs: 176

Target asset (AAPL) dividend yield:
  Mean: 53.90%
  Std: 7.60%

Replication portfolio dividend yield:
  Mean: 189.68%
  Std: 45.52%

Forecast Differences:
  Mean difference: -135.78%
  Std of difference: 44.63%
  Max absolute difference: 229.05%

Correlation between forecasts: 0.1988

Regression: Replication Yield = α + β × Target Yield
  Intercept (α): 1.2549
  Slope (β): 1.1909  [should be ≈1 for consistency]
  R²: 0.0395

NO-ARBITRAGE IMPLICATIONS

1. Replication Quality:
   → Poor replication (TE = 20.70%)
   → Targe

,Stock,Weight
0,ADBE,0.0043
8,TER,0.0041
5,NXPI,0.0037
2,GOOG,0.0025
3,MCO,0.0025
7,ROP,0.0023
6,QCOM,0.0018
1,ADI,0.0017
10,WDAY,0.0008
9,TSLA,0.0005


### 5.2. Arbitrage

Do your forecasts imply arbitrage?

It doesn't, as the replication portfolio has high tracking error. No riskless returns.